Autor: Antonio Fernández Salcedo / [Hugging Face](https://huggingface.co/Antonio49)

Trabajando: Canal de Isabel II (https://www.canaldeisabelsegunda.es)

Formación: Grado Ingeniería Informática (https://www.uoc.edu/es)

Aplicación desarrollada para TFG. Inteligencia Artificial.

Contacto: afernandezsalc@uoc.edu         

# Trabajar con conjuntos de datos de preguntas y respuestas etiquetadas y modelos de Hugging Face

### Cargando y explorando un conjunto de datos de preguntas y respuestas

1. Cargaremos un conjunto de datos de preguntas y respuestas del Canal de Isabel II.
2. Extraeremos información del conjunto de datos y realizaremos un fine-tuning.


### El método de fine-tuning se basará en:

1. Ajustar **Hiperparametros** esenciales.

> **lr = 1e-5.**  Tasa de aprendizaje: tamaño del paso de optimización *(size of optimization step).*

> **batch_size = 8.**  Tamaño del lote: número de ejemplos procesados por paso de optimización*(No.of.examples processed per optimization step).*

> **num_epochs = 10.**  Número de épocas: número de veces que el modelo atraviesa los datos de entrenamiento*(No.of.times the model runs through training data).*

> **weight_decay = 0.006.**   Decaimiento de peso para evitar sobreajuste.

2. Agregar capas adicionales que se integrarán en la arquitectura existente del modelo BERT para ajustarse mejor a la tarea específica de preguntas y respuestas.

3. *Linearly decaying Learning rate with Restart Algorithm* **(LoRA) con el Optimizador SGD** (*Stochastic Gradient Descent*).

4. *Linearly decaying Learning rate with Restart Algorithm*  **(LoRA) con el Optimizador Adam** (*Adaptive Moment Estimation*).

5. *Linearly decaying Learning rate with Restart Algorithm*  **(LoRA) con el Optimizador AdamW** (*Adam with Weight Decay*).




# Requisitos de hardware

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue May 14 14:55:01 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

Aquí estamos usando una Tesla **T4 GPU**. Y soporta un entrenamiento de bastantes epocas.

Si `torch.cuda.is_available()` devuelve False, significa que no se detectó ninguna GPU disponible en tu sistema. Hay algunas razones comunes por las que esto podría suceder:

1. **Falta de GPU**: Si estás ejecutando el código en una máquina que no tiene una tarjeta gráfica compatible con CUDA, `torch.cuda.is_available()` devolverá False.

2. **Configuración incorrecta**: Asegúrate de que los controladores de la GPU estén instalados correctamente y que CUDA esté configurado adecuadamente en tu sistema.

3. **Entorno de ejecución**: Si estás ejecutando el código en un entorno como Google Colab o Kaggle, es posible que no se te asigne una GPU. Deberías verificar la configuración del entorno o intentar cambiar a uno que ofrezca aceleración por GPU.

4. **Limitaciones de acceso**: En algunos casos, el acceso a la GPU puede estar restringido debido a políticas de seguridad o privilegios de usuario. Asegúrate de tener los permisos adecuados para acceder a la GPU.

Si ninguna de estas soluciones resuelve el problema, puede haber problemas más específicos relacionados con la configuración del sistema.

In [ ]:
import torch
torch.cuda.is_available()

True

# RAM

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('El tiempo de ejecución tiene {:.1f} gigabytes de RAM disponible\n'.format(ram_gb))

if ram_gb < 12:
  print('Para habilitar un tiempo de ejecución de RAM alto, seleccione > tiempo de ejecución "Cambiar tipo de tiempo de ejecución"')
else:
  print('¡Estas utilizando un tiempo de ejecución de alta RAM!')

El tiempo de ejecución tiene 13.6 gigabytes de RAM disponible

¡Estas utilizando un tiempo de ejecución de alta RAM!


# Instalación de Transformers y Datasets

In [ ]:
# Instalacion
! pip install datasets transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 11.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.wh

# Importación de librerías

In [ ]:
import json
from google.colab import drive
from datasets import Dataset
from datasets import DatasetDict
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import TrainingArguments, Trainer
from transformers import DefaultDataCollator
from transformers import pipeline
from tqdm import tqdm
from IPython.core.display import HTML
from scipy.spatial.distance import cosine

# Montado en /content/drive

Cargar un conjunto de datos de preguntas_respuestas.json especifico del **Canal de Isabel II**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import json

# Abrir el archivo JSON en modo lectura
with open('/content/drive/MyDrive/TFG/preguntas_respuestas.json', 'r') as file:
    # Cargar el contenido del archivo JSON en una variable Python
    data = json.load(file)

# Ahora, la variable 'data' contiene los datos del archivo JSON
print(data)


[{'id': '100', 'title': 'Preguntas frecuentes: Averías', 'context': 'En primer lugar, averigua si la falta de agua afecta a todos los puntos de agua de tu casa. Si no, pregunta a algún vecino si tiene agua. En caso de que sí tenga, comprueba que la llave de paso interior de tu vivienda esté completamente abierta y verifica que el contador o el equipo de medición tenga las dos llaves de paso abiertas (es decir, giradas completamente a la izquierda). En la batería de contadores hay un plano donde podrás encontrar la ubicación de tu contador. Si después de haber hecho estas comprobaciones sigues sin agua, ponte en contacto con nosotros.', 'question': '¿Qué debo hacer si no tengo agua en mi casa?', 'answers': {'text': ['En primer lugar, averigua si la falta de agua afecta a todos los puntos de agua de tu casa. Si no, pregunta a algún vecino si tiene agua'], 'answer_start': [0]}}, {'id': '101', 'title': 'Preguntas frecuentes: Averías', 'context': 'En primer lugar, averigua si la falta de ag

# Enumerar los metadatos y el contenido del conjunto de datos

In [ ]:
from datasets import Dataset

# Suponiendo que 'data' es la lista de datos que cargaste del archivo JSON
# Define las claves esperadas en el conjunto de datos
keys = ['id', 'title', 'context', 'question', 'answers']

# Convierte la lista de datos en un diccionario con las claves esperadas
data_dict = {key: [item[key] for item in data] for key in keys}

# Crea el conjunto de datos a partir del diccionario
dataset = Dataset.from_dict(data_dict)

# Imprime el conjunto de datos
print(dataset)

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 506
})


In [ ]:
from datasets import DatasetDict

In [ ]:
# Convertir la lista de diccionarios en un DatasetDict
dataset_diccionario = DatasetDict({"train": dataset})

# Verificar el tipo de dataset
print(type(dataset_diccionario))

<class 'datasets.dataset_dict.DatasetDict'>


In [ ]:
dataset_diccionario

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 506
    })
})

# Creación de un conjunto de entrenamiento y prueba a partir de un conjunto de datos

In [ ]:
# Dividir el conjunto de datos(diccionario) en conjuntos de datos de entrenamiento y validación
data = dataset_diccionario['train'].train_test_split(train_size=0.8, seed=42)

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 404
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 102
    })
})

In [ ]:
# Renombrar el valor predeterminado "test" por "val"
data['validation'] = data.pop("test")

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 404
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 102
    })
})

In [ ]:
# Mostrar el dataset
dataset

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 506
})

In [ ]:
import pandas as pd

# Obtener el conjunto de datos
data2 = dataset

# Convertir a DataFrame
df2 = pd.DataFrame(data2)

# Visualizar el DataFrame
print(df2.head(19))

     id                          title  \
0   100  Preguntas frecuentes: Averías   
1   101  Preguntas frecuentes: Averías   
2   102  Preguntas frecuentes: Averías   
3   103  Preguntas frecuentes: Averías   
4   104  Preguntas frecuentes: Averías   
5   105  Preguntas frecuentes: Averías   
6   106  Preguntas frecuentes: Averías   
7   107  Preguntas frecuentes: Averías   
8   108  Preguntas frecuentes: Averías   
9   109  Preguntas frecuentes: Averías   
10  110  Preguntas frecuentes: Averías   
11  111  Preguntas frecuentes: Averías   
12  112  Preguntas frecuentes: Averías   
13  113  Preguntas frecuentes: Averías   
14  114  Preguntas frecuentes: Averías   
15  115  Preguntas frecuentes: Averías   
16  116  Preguntas frecuentes: Averías   
17  117  Preguntas frecuentes: Averías   
18  118  Preguntas frecuentes: Averías   

                                              context  \
0   En primer lugar, averigua si la falta de agua ...   
1   En primer lugar, averigua si la falta de 

In [ ]:
df2.head(3)

,id,title,context,question,answers
0,100,Preguntas frecuentes: Averías,"En primer lugar, averigua si la falta de agua ...",¿Qué debo hacer si no tengo agua en mi casa?,"{'answer_start': [0], 'text': ['En primer luga..."
1,101,Preguntas frecuentes: Averías,"En primer lugar, averigua si la falta de agua ...",¿Cuáles son las acciones recomendadas si carez...,"{'answer_start': [0], 'text': ['En primer luga..."
2,102,Preguntas frecuentes: Averías,"En primer lugar, averigua si la falta de agua ...",¿Qué pasos debo seguir si no tengo suministro ...,"{'answer_start': [0], 'text': ['En primer luga..."


# Indexar y segmentar el conjunto de datos

In [ ]:
# Consigue el registro 1 del conjunto de datos
dataset[1]

{'id': '101',
 'title': 'Preguntas frecuentes: Averías',
 'context': 'En primer lugar, averigua si la falta de agua afecta a todos los puntos de agua de tu casa. Si no, pregunta a algún vecino si tiene agua. En caso de que sí tenga, comprueba que la llave de paso interior de tu vivienda esté completamente abierta y verifica que el contador o el equipo de medición tenga las dos llaves de paso abiertas (es decir, giradas completamente a la izquierda). En la batería de contadores hay un plano donde podrás encontrar la ubicación de tu contador. Si después de haber hecho estas comprobaciones sigues sin agua, ponte en contacto con nosotros.',
 'question': '¿Cuáles son las acciones recomendadas si carezco de agua en mi hogar?',
 'answers': {'answer_start': [0],
  'text': ['En primer lugar, averigua si la falta de agua afecta a todos los puntos de agua de tu casa. Si no, pregunta a algún vecino si tiene agua']}}

In [ ]:
# Obtener los primeros 13 registros del conjunto de datos
test_last = dataset[0:12]

In [ ]:
# Crear un DataFrame de Pandas
pd.DataFrame(test_last)

,id,title,context,question,answers
0,100,Preguntas frecuentes: Averías,"En primer lugar, averigua si la falta de agua ...",¿Qué debo hacer si no tengo agua en mi casa?,"{'answer_start': [0], 'text': ['En primer luga..."
1,101,Preguntas frecuentes: Averías,"En primer lugar, averigua si la falta de agua ...",¿Cuáles son las acciones recomendadas si carez...,"{'answer_start': [0], 'text': ['En primer luga..."
2,102,Preguntas frecuentes: Averías,"En primer lugar, averigua si la falta de agua ...",¿Qué pasos debo seguir si no tengo suministro ...,"{'answer_start': [0], 'text': ['En primer luga..."
3,103,Preguntas frecuentes: Averías,"En primer lugar, averigua si la falta de agua ...","Si no tengo acceso a agua en mi vivienda, ¿qué...","{'answer_start': [0], 'text': ['En primer luga..."
4,104,Preguntas frecuentes: Averías,"En primer lugar, averigua si la falta de agua ...",¿Cómo debo actuar si enfrento una situación en...,"{'answer_start': [0], 'text': ['En primer luga..."
5,105,Preguntas frecuentes: Averías,"En primer lugar, averigua si la falta de agua ...",¿Qué debo revisar si tengo falta de agua en mi...,"{'answer_start': [163], 'text': ['Comprueba qu..."
6,106,Preguntas frecuentes: Averías,"En primer lugar, averigua si la falta de agua ...",¿Qué debo verificar si no tengo agua en mi casa?,"{'answer_start': [163], 'text': ['Comprueba qu..."
7,107,Preguntas frecuentes: Averías,"En primer lugar, averigua si la falta de agua ...","Si carezco de agua en mi vivienda, ¿qué elemen...","{'answer_start': [163], 'text': ['Comprueba qu..."
8,108,Preguntas frecuentes: Averías,"En primer lugar, averigua si la falta de agua ...","Si enfrento escasez de agua en mi hogar, ¿qué ...","{'answer_start': [163], 'text': ['Comprueba qu..."
9,109,Preguntas frecuentes: Averías,"En primer lugar, averigua si la falta de agua ...",¿Cuáles son las cosas básicas que debo inspecc...,"{'answer_start': [163], 'text': ['Comprueba qu..."


In [ ]:
# Acceder a los primeros 5 ejemplos de el conjunto dedel diccionario
data_train = data['train'][:5]

# Imprimir los primeros 5 ejemplos del conjunto de datos
for example in data_train:
    print(example, ":", data_train[example])

id : ['333', '1052', '711', '311', '933']
title : ['Bonificaciones', 'Contratación y cambio de titularidad', 'Gestiones y contratos', 'Bonificaciones', 'Facturación']
context : ['Pensión de viudedad.  Perceptores de una pensión por viudedad con renta total inferior a 14.000 euros anuales. La bonificación será del 50 % del importe de la cuota de servicio fija. Documentación que debes aportar por pensión de viudedad: Certificado, emitido por los órganos o entidades competentes, que acredite la condición de perceptor de una pensión de viudedad y el importe de la misma. Fotocopia del Documento Nacional de Identidad del solicitante. Solicitud de bonificación de la factura de consumo de agua, marcando la casilla de declaración responsable de que sus ingresos totales, incluida la pensión de viudedad, no superan el importe establecido.', 'El Decreto 3068/75, de 31 de octubre, establece que en la contratación del suministro se pueden facturar los siguientes conceptos: 1. Cuota de red: correspon

# Cargar el modelo BERT previamente entrenado

Para las tareas de respuesta a preguntas, un modelo previamente entrenado comúnmente utilizado es **BERT** (*Bidirectional Encoder Representations from Transformer*/Representaciones de codificador bidireccional de Transformers).

**Hugging Face** proporciona una interfaz fácil de usar para cargar este modelo junto con su tokenizador.

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

#Cargar el tokenizerador y el  modelo
tokenizer = AutoTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased-finetuned-qa-mlqa')
tokenizer


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/539 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/486k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

BertTokenizerFast(name_or_path='dccuchile/bert-base-spanish-wwm-uncased-finetuned-qa-mlqa', vocab_size=31002, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	5: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

Los datos proporcionados por el tokenizador tienen el siguiente significado:

1. `name_or_path`: Es la ruta o el nombre del modelo que el tokenizador está utilizando o cargando. En este caso, parece ser la ruta de un modelo almacenado en Google Drive.

2. `vocab_size`: Es el tamaño del vocabulario del tokenizador, es decir, el número total de tokens únicos que el tokenizador puede manejar.

3. `model_max_length`: Es la longitud máxima de secuencia admitida por el modelo. Para secuencias más largas, el tokenizador truncará o dividirá la secuencia de entrada según sea necesario.

4. `is_fast`: Indica si el tokenizador es una versión rápida (True) o no (False).

5. `padding_side`: Indica el lado del texto al que se aplicará el relleno. En este caso, el relleno se aplica en el lado derecho.

6. `truncation_side`: Indica el lado del texto al que se aplicará la truncación si la secuencia excede model_max_length. En este caso, la truncación se aplica en el lado derecho.

7. `special_tokens`: Es un diccionario que contiene tokens especiales y sus representaciones específicas. Estos tokens especiales se utilizan para indicar el comienzo y el final de una secuencia, así como para el relleno, el enmascaramiento y tokens desconocidos. Por ejemplo, **[PAD]** se utiliza para el relleno, **[CLS]** para el inicio de la secuencia, **[SEP]** para separar secuencias y **[MASK]** para el enmascaramiento.

8. `added_tokens_decoder`: Es un diccionario que contiene tokens adicionales agregados al vocabulario original del modelo. Cada token tiene un índice asignado y sus atributos correspondientes, como si es especial, si es un token único o si necesita ser normalizado.

In [ ]:
# Cargar el  modelo
model = AutoModelForQuestionAnswering.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased-finetuned-qa-mlqa')
model

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/437M [00:00<?, ?B/s]

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elem

# *Tokenize the Data* (Tokenizar los datos)

Los datos del dataset deben tokenizarse utilizando el tokenizador del modelo seleccionado.

La función `prepare_train_features` toma como entrada un conjunto de ejemplos de entrenamiento (diccionarios) y los transforma en un conjunto de características tokenizadas y etiquetadas para entrenar un modelo de respuesta a preguntas (QA). Las características tokenizadas se basan en el modelo de lenguaje pre-entrenado `tokenizer`. Los pasos que realiza la función:

1. **Tokenización y truncamiento**:

*   La función tokeniza la pregunta y el contexto de cada ejemplo utilizando el tokenizer pre-entrenado.
*   Se aplica un truncamiento "solo al segundo" para mantener la pregunta completa y truncar el contexto si es necesario.
*   La longitud máxima de la secuencia se establece en 512 tokens.
*   Se utiliza un "paso" de 128 para generar múltiples características a partir de un contexto largo, superponiendo un poco cada característica.

2. **Mapeo de características y offsets**:

*   Se crea un mapeo de "desbordamiento a muestra" para identificar el ejemplo original de cada característica generada a partir de un contexto largo.
*   Se crea un mapeo de "offsets" para obtener la posición del carácter original para cada token en la secuencia tokenizada.

3. **Etiquetado de ejemplos**:

*   Se inicializan las listas `start_positions` y `end_positions` para almacenar las posiciones de inicio y fin de las respuestas en la secuencia tokenizada.
*   Para cada ejemplo:

> *   Se verifica si hay respuestas en el ejemplo.
*   Si no hay respuestas, se establece la respuesta como el token CLS (índice especial).
*   Si hay respuestas:

a. Se obtienen las posiciones de inicio y fin del carácter de la respuesta en el texto original.

b. Se calcula el índice de inicio y fin del token de la respuesta en la secuencia tokenizada.

c. Se comprueba si la respuesta está dentro del fragmento actual.

d. Si la respuesta está fuera del fragmento, se establece la respuesta como el token CLS.

e. Si la respuesta está dentro del fragmento, se ajustan los índices de inicio y fin del token para abarcar la respuesta.

4. **Salida**:

La función devuelve un diccionario con las características tokenizadas y etiquetadas:

1. `input_ids`: codificación numérica de los tokens `context`+`question`
2. `token_type_ids`: secuencia (del mismo tamaño de `input_ids`) indicando la porción correspondiente a `context` (codificada con 1s), `answer` (codificada con 0s) y "padding" (también con 0s)
3. `attention_mask`: la máscara atencional (del mismo tamaño de `input_ids`) indicando la porción correspondiente a `context`+`answer` (codificada con 1s) y la porción que contiene "padding" (codificada con 0s)
4. `start_positions`: la posición del token de inicio de la respuesta, con respecto a la secuencia `input_ids`
5. `end_positions`: la posición del token de finalización de la respuesta, con respecto a la secuencia `input_ids`

En resumen, la función `prepare_train_features` transforma ejemplos de entrenamiento en características tokenizadas y etiquetadas que se pueden utilizar para entrenar un modelo de respuesta a preguntas.

Aquí hay algunos puntos adicionales a tener en cuenta:

*   La función está diseñada para trabajar con el formato de datos específico del conjunto de datos utilizado.

*   La característica `padding` en la función de tokenización: En la función prepare_train_features, se ha definido `padding="max_length"`. Esto indica que todas las secuencias de entrada (tanto las preguntas como los contextos) seran rellenadas con tokens de padding para que tengan la misma longitud máxima (max_length). Este es un paso importante para asegurarse de que todas las secuencias en un lote tengan la misma longitud.






In [ ]:
# Definir la función de tokenización y procesamiento de los datos
def prepare_train_features(examples):
    # Tokenizamos nuestros ejemplos con truncamiento y padding, pero mantenemos los desbordamientos usando un paso.
    # Esto resulta en que un ejemplo pueda dar varias características cuando un contexto es largo,
    # cada una de esas características con un contexto que se superpone un poco al contexto de la característica anterior.
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",  # Truncar contexto, no la pregunta
        max_length=512,            # Longitud máxima de la secuencia
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Dado que un ejemplo podría darnos varias características si tiene un contexto largo,
    # necesitamos un mapeo de una característica a su ejemplo correspondiente. Esta clave nos da justo eso.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # Los mapeos de desplazamiento nos darán un mapa desde el token hasta la posición del carácter en el contexto original.
    # Esto nos ayudará a calcular las posiciones de inicio y fin.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Etiquetemos esos ejemplos
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # Etiquetaremos respuestas imposibles con el índice del token CLS.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Capturamos la secuencia correspondiente a ese ejemplo (para saber qué es el contexto y qué es la pregunta).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # Un ejemplo puede dar varios fragmentos, este es el índice del ejemplo que contiene este fragmento de texto.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # Si no se dan respuestas, establecer el índice cls como respuesta.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Índice de inicio/fin de caracteres de la respuesta en el texto.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Índice de inicio del token del fragmento actual en el texto.
            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1

            # Índice de fin del token del fragmento actual en el texto.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            # Detectar si la respuesta está fuera del fragmento (en cuyo caso esta característica se etiqueta con el índice CLS).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # De lo contrario, mover el índice de inicio del token y el índice de fin del token a los dos extremos de la respuesta.
                # Nota: podríamos ir después del último desplazamiento si la respuesta es la última palabra (caso límite).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples


In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 404
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 102
    })
})

In [ ]:
example = data['train'][8]
for key in example:
    print(key, ":", example[key])

id : 940
title : Facturación
context : Según establece el Reglamento para el Servicio y Distribución de las Aguas de Canal de Isabel II, (Decreto 2922/1975, de 31 de octubre) podríamos suspender el suministro de agua si transcurridos 30 días naturales desde la emisión de la factura, esta no ha sido abonada. El corte del suministro implicaría continuar facturando las cuotas de servicio. El restablecimiento del servicio se realizará una vez liquidada la deuda, así como el importe del restablecimiento. Transcurridos tres meses sin que se haya producido el pago, Canal puede resolver el contrato y proceder a la condena de la acometida (retirada de la instalación). Para volver a tener agua habrá que realizar una nueva contratación.
question : ¿Qué implicaría el corte del suministro de agua?
answers : {'answer_start': [270], 'text': ['El corte del suministro implicaría continuar facturando las cuotas de servicio']}


In [ ]:
# Aplicar la función a los datos
tokenized_datasets = data.map(prepare_train_features, batched=True, remove_columns=data["train"].column_names)


Map:   0%|          | 0/404 [00:00<?, ? examples/s]

Map:   0%|          | 0/102 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 404
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 102
    })
})

Cuando tokenizamos el dataset, obtenemos varios vectores que representan la información del texto en un formato adecuado. El significado de cada vector:

1. **input_ids**:

*   Contiene los identificadores (IDs) enteros asignados a cada token en la secuencia de entrada después de la tokenización.
*   Cada token en el texto original se mapea a un ID único basado en el vocabulario del modelo (un diccionario que vincula palabras a números).
*   Este vector tiene la misma longitud que la secuencia tokenizada.
*   Ejemplo: Si la tercera palabra en tu texto original es "Antonio" y su ID en el vocabulario es 24, entonces input_ids[2] sería 24.

2. **token_type_ids**:

*   Este vector es opcional y depende del modelo y la tarea específica.
*   Se utiliza en algunos modelos de Transformers para diferenciar entre tokens de diferentes segmentos en la entrada.
*   Por ejemplo, en nuestro caso "[CLS] (indicando clasificación) al principio y [SEP] (indicando separación) entre las oraciones".

3. **attention_mask**:

*   Este vector es un vector binario (contiene 0s y 1s) de la misma longitud que `input_ids`.
*   Indica al modelo qué partes de la secuencia de entrada son relevantes para la tarea y cuáles se deben ignorar.
*   Los valores 1 corresponden a tokens válidos que el modelo debe atender, mientras que los valores 0 corresponden a tokens que se deben ignorar (por ejemplo, *padding*).
*   Se utiliza para manejar secuencias de diferentes longitudes y asegurarse de que el modelo no preste atención a tokens de relleno.

4. **start_positions**:

*   Este vector es opcional y depende de la tarea específica (generalmente, tareas de respuesta a preguntas).
*   Contiene el índice del primer token en la secuencia de entrada que corresponde al inicio de la respuesta prevista.
*   Si no estás realizando una tarea de respuesta a preguntas, es posible que este vector no esté presente.

5. **end_positions**:

*   Este vector es opcional y depende de la tarea específica (generalmente, tareas de respuesta a preguntas).
*   Contiene el índice del último token en la secuencia de entrada que corresponde al final de la respuesta prevista.
*   Si no estás realizando una tarea de respuesta a preguntas, es posible que este vector no esté presente.

**Resumen:**

1. input_ids: Números enteros que representan tokens individuales en el texto.
2. token_type_ids: Distingue segmentos en la entrada (depende del modelo).
3. attention_mask: Indica qué tokens son relevantes para la tarea (valores 0 y 1).
4. start_positions: Índice del primer token de la respuesta prevista.
5. end_positions: Índice del último token de la respuesta prevista.

Estos vectores, forman la base del dataset tokenizado que se utilizará para entrenar o realizar inferencia con su modelo preentrenado.

Ejemplo: Vemos el vector tokenizado de la linea 2 del conjunto train, de la pregunta con el contexto.

In [ ]:
print(tokenized_datasets['train'][2]['input_ids'])

[4, 1063, 1140, 6743, 1555, 2860, 1012, 3170, 1035, 2503, 1048, 1285, 20560, 1096, 1054, 1581, 8563, 1009, 1032, 2599, 15417, 1059, 5, 1096, 1054, 1669, 8563, 1009, 1032, 2599, 15417, 1862, 4267, 1085, 3307, 20914, 995, 1098, 1008, 5664, 1048, 9002, 995, 13511, 1044, 8130, 1019, 7683, 1009, 3531, 1151, 5403, 11173, 1040, 4267, 20914, 16954, 1048, 19670, 13644, 1040, 14115, 1008, 1129, 1008, 5664, 1048, 2246, 20560, 995, 3586, 2246, 20560, 1019, 11606, 4057, 20560, 20945, 1019, 4267, 20800, 1154, 1009, 20560, 1019, 9903, 3022, 9917, 1040, 7683, 1009, 3531, 1035, 1032, 17440, 9732, 1008, 1306, 1008, 4574, 1012, 5826, 30958, 1040, 21183, 995, 10228, 1032, 3305, 1009, 1032, 4967, 7810, 1040, 7013, 1032, 7810, 1009, 1203, 20225, 1008, 1002, 1008, 1811, 1916, 1009, 20914, 995, 4152, 1032, 24171, 1009, 1044, 4881, 1019, 7683, 1009, 3531, 1035, 1032, 2599, 15417, 1019, 4142, 1091, 5104, 1097, 2857, 2613, 6166, 1035, 1091, 2599, 1019, 17047, 2246, 12954, 1019, 16403, 14889, 30958, 1019, 10228, 

In [ ]:
print(tokenized_datasets['train'][2]['token_type_ids'])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
print(tokenized_datasets['train'][2]['attention_mask'])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
print(tokenized_datasets['train'][2]['start_positions'])

70


In [ ]:
print(tokenized_datasets['train'][2]['end_positions'])

95


# Del conjunto train, obtenemos la pregunta, la respuesta a cada pregunta y la respuesta etiqueta dentro del contexto.

In [ ]:
row = 1
for example in data['train']:
  print(f"{row}: {example['question']}")
  row += 1

1: ¿Cuáles son los grupos que reciben beneficios en la pensión de viudedad según el Canal de Isabel II?
2: ¿Qué gastos incluye la contratación de un nuevo suministro de agua?
3: ¿Qué opciones puedo llevar a cabo en relación con mis facturas si no soy usuario de la Oficina Virtual?
4: ¿Quiénes son elegibles para recibir beneficios del Canal de Isabel II?
5: ¿En qué casos puede Canal de Isabel II cortar el suministro de agua?
6: ¿Qué pasos debo seguir para aumentar el caudal de agua en mi vivienda?
7: ¿Qué pasa si no cambio el nombre del contrato del agua después de comprar una casa?
8: ¿Qué pasos debo seguir si no tengo suministro de agua en casa?
9: ¿Qué implicaría el corte del suministro de agua?
10: ¿En qué consiste la cuota de servicio en la factura?
11: ¿Cómo se compone la factura del agua?
12: ¿Qué información debes proporcionar en todas las comunicaciones?
13: ¿Hay alguna app para saber si no va a haber agua?
14: ¿Dónde puedo encontrar el servicio de notificaciones de incidencias

In [ ]:
row = 1
for example in data['train']:
  print(f"{row}: {example['answers']['text'][0]}")
  start = example['answers']['answer_start'][0]
  print(f"  {example['context'][start:(start + len(example['answers']['text'][0]))]}")
  row += 1

1: Perceptores de una pensión por viudedad con renta total inferior a 14.000 euros anuales
  Perceptores de una pensión por viudedad con renta total inferior a 14.000 euros anuales
2: 1. Cuota de red: corresponde al importe de las obras de adaptación y mejora de la red existente. Este concepto se facturará como norma general para los nuevos suministros solicitados. 2. Cuota de enganche: se trata de los costes derivados de la ejecución de la obra, tales como materiales y mano de obra. 3. Anticipo de consumo: es una cantidad que cubre el importe de la facturación del suministro de agua
  1. Cuota de red: corresponde al importe de las obras de adaptación y mejora de la red existente. Este concepto se facturará como norma general para los nuevos suministros solicitados. 2. Cuota de enganche: se trata de los costes derivados de la ejecución de la obra, tales como materiales y mano de obra. 3. Anticipo de consumo: es una cantidad que cubre el importe de la facturación del suministro de agua


In [ ]:
row = 1
for example in tokenized_datasets['train']:
  answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(example['input_ids'][example['start_positions']:example['end_positions']+1]))
  print(f'{row}:  answer = {answer}')
  row += 1

1:  answer = perceptores de una pensión por viudedad con renta total inferior a 14. 000 euros anuales
2:  answer = 1. cuota de red : corresponde al importe de las obras de adaptación y mejora de la red existente. este concepto se facturará como norma general para los nuevos suministros solicitados. 2. cuota de enganche : se trata de los costes derivados de la ejecución de la obra, tales como materiales y mano de obra. 3. anticipo de consumo : es una cantidad que cubre el importe de la facturación del suministro de agua
3:  answer = pagar tus facturas, visualizar facturas electrónicas, realizar simulaciones de facturas, solicitar bonificaciones y darte de alta en la factura electrónica
4:  answer = perceptores de una pensión por viudedad. familias o viviendas numerosas
5:  answer = podríamos suspender el suministro de agua si transcurridos 30 días naturales desde la emisión de la factura, esta no ha sido abonada
6:  answer = comprueba que la llave de paso interior de tu casa esté comple

# Del conjunto validation, obtenemos la pregunta, la respuesta a cada pregunta y la respuesta etiqueta dentro del contexto.

In [ ]:
row = 1
for example in data['validation']:
  print(f"{row}: {example['question']}")
  row += 1

1: ¿Cómo se detallan los cargos por consumo de agua en la factura de Canal de Isabel II?
2: ¿Hay algún horario para usar el teléfono gratis?
3: ¿Puedo fraccionar el pago de mi factura de agua de Canal de Isabel II?
4: ¿Se puede recibir notificaciones sobre interrupciones en el suministro de agua?
5: ¿Cómo puedo solucionar la falta de presión de agua si la llave de paso interior está abierta?
6: ¿Qué conceptos hay en la factura?
7: No tengo presión de agua y ya revisé la llave de paso, ¿a qué se puede deber?
8: ¿Dónde envío una carta?
9: ¿Existen otras pistas que me ayuden a identificar una fuga de agua en las tuberías de riego?
10: ¿Por qué la duración del contrato provisional de agua está limitada por la licencia de obra?
11: ¿Hay alguna app para recibir avisos sobre cortes de agua?
12: ¿Cómo se reparten los costes de aducción, distribución, depuración y alcantarillado en la factura del agua?
13: ¿Qué requisitos hay para darme de alta en el agua con Canal de Isabel II?
14: ¿Dónde pued

In [ ]:
row = 1
for example in data['validation']:
  print(f"{row}: {example['answers']['text'][0]}")
  start = example['answers']['answer_start'][0]
  print(f"  {example['context'][start:(start + len(example['answers']['text'][0]))]}")
  row += 1

1: 1.Estimación de consumos (EST): este método se utiliza cuando no es posible acceder al contador en la fecha en que debe realizarse la lectura. Su cálculo se efectúa de acuerdo con el consumo medio diario de los dos periodos análogos precedentes y, en caso de disponerse solo del histórico de consumos de un año, la estimación se realizará de acuerdo con el consumo medio diario del periodo análogo precedente. El consumo facturado de esta forma se considera a cuenta. Por tanto, una vez que se tome la lectura real del contador, los metros cúbicos facturados de esta forma se descontarán del consumo realizado. 2.Evaluación de consumos (EV): para facturar los metros cúbicos mediante este método, se debe dar la circunstancia de que, aun teniendo acceso al contador, no se pueda recoger la lectura porque exista una anomalía que requiera su sustitución. El cálculo del consumo se realiza de forma análoga a la estimación. 3.Diferencia de lecturas del aparato de medida (DFI): diferencia de la lect

In [ ]:
row = 1
for example in tokenized_datasets['validation']:
  answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(example['input_ids'][example['start_positions']:example['end_positions']+1]))
  print(f'{row}:  answer = {answer}')
  row += 1

1:  answer = 1. estimación de consumos ( est ) : este método se utiliza cuando no es posible acceder al contador en la fecha en que debe realizarse la lectura. su cálculo se efectúa de acuerdo con el consumo medio diario de los dos periodos análogos precedentes y, en caso de disponerse solo del histórico de consumos de un año, la estimación se realizará de acuerdo con el consumo medio diario del periodo análogo precedente. el consumo facturado de esta forma se considera a cuenta. por tanto, una vez que se tome la lectura real del contador, los metros cúbicos facturados de esta forma se descontarán del consumo realizado. 2. evaluación de consumos ( ev ) : para facturar los metros cúbicos mediante este método, se debe dar la circunstancia de que, aun teniendo acceso al contador, no se pueda recoger la lectura porque exista una anomalía que requiera su sustitución. el cálculo del consumo se realiza de forma análoga a la estimación. 3. diferencia de lecturas del aparato de medida ( dfi ) :

# Vector de tokens de la pregunta y el contexto de todo el conjunto train.

In [ ]:
row = 1
for example in tokenized_datasets['train']:
  print(f"{row}:  pregunta + contexto = {example['input_ids']}")
  row += 1

1:  pregunta + contexto = [4, 1063, 8299, 1318, 1067, 3145, 1041, 9868, 6561, 1035, 1032, 11364, 1009, 1690, 4883, 1189, 2241, 1039, 5413, 1009, 10178, 2836, 1059, 5, 11364, 1009, 1690, 4883, 1189, 1008, 11281, 1953, 1009, 1091, 11364, 1076, 1690, 4883, 1189, 1048, 7425, 2218, 6244, 1012, 1098, 1003, 1008, 2371, 6161, 9482, 1008, 1032, 3022, 3225, 2119, 1081, 3092, 991, 1081, 8577, 1009, 1032, 13431, 1009, 2809, 11239, 1008, 8288, 1041, 3525, 13750, 1076, 11364, 1009, 1690, 4883, 1189, 995, 10453, 1019, 14778, 1076, 1067, 4887, 1068, 6417, 9429, 1019, 1041, 10913, 1043, 1032, 5145, 1009, 11281, 1175, 1009, 1091, 11364, 1009, 1690, 4883, 1189, 1040, 1039, 8577, 1009, 1032, 2506, 1008, 30083, 6613, 1081, 3878, 1954, 1009, 6635, 1081, 20196, 1008, 5705, 1009, 3022, 3225, 1009, 1032, 17440, 1009, 5826, 1009, 2326, 1019, 23836, 1032, 28027, 1009, 3421, 5148, 1009, 1041, 1233, 4614, 15512, 1019, 6584, 1032, 11364, 1009, 1690, 4883, 1189, 1019, 1054, 25012, 1039, 8577, 5400, 1008, 5, 1, 1, 1,

# Vector de tokens de la pregunta y el contexto de todo el conjunto validation.

In [ ]:
row = 1
for example in tokenized_datasets['validation']:
  print(f"{row}:  pregunta + contexto = {example['input_ids']}")
  row += 1

1:  pregunta + contexto = [4, 1063, 1475, 1057, 25900, 1029, 1067, 6800, 1076, 5826, 1009, 2326, 1035, 1032, 17440, 1009, 5413, 1009, 10178, 2836, 1059, 5, 1039, 2989, 1097, 1039, 2809, 1040, 5370, 1009, 1085, 5257, 1009, 5413, 1009, 10178, 2836, 1147, 7366, 3357, 10437, 989, 1220, 998, 31001, 1019, 1009, 3156, 1009, 2967, 1135, 5626, 2909, 5649, 1097, 17440, 30960, 1039, 5826, 1009, 2326, 995, 1098, 1008, 15263, 1009, 5826, 30958, 1147, 1078, 1135, 995, 1277, 6385, 1057, 6174, 1351, 1054, 1028, 2368, 8422, 1074, 20225, 1035, 1032, 3305, 1035, 1041, 1899, 14937, 1032, 7810, 1008, 1069, 11439, 1057, 26264, 1009, 1724, 1048, 1039, 5826, 2079, 5092, 1009, 1067, 1411, 19322, 22817, 1725, 14153, 1040, 1019, 1035, 2053, 1009, 11980, 1182, 1628, 1081, 7149, 1009, 5826, 30958, 1009, 1044, 1730, 1019, 1032, 15263, 1057, 18895, 1009, 1724, 1048, 1039, 5826, 2079, 5092, 1081, 6083, 22817, 1110, 17678, 1008, 1039, 5826, 17440, 1050, 1009, 1149, 1795, 1057, 4345, 1012, 1946, 1008, 1076, 1850, 1019,

# *Fine-Tuning del  Modelo* (Ajustar el modelo)

Configuramos los argumentos para el entrenamiento del modelo utilizando la biblioteca Transformers. Estos argumentos incluyen la estrategia de evaluación, la tasa de aprendizaje, el tamaño del lote de entrenamiento y evaluación, el número de épocas de entrenamiento y el decaimiento de peso. Estos parámetros son esenciales para controlar cómo se realizará el entrenamiento del modelo.

Para asegurarse de que se registre la pérdida de entrenamiento correctamente, agregamos el argumento `report_to` en los `TrainingArguments` y se establece en `"all"` para que se informen todos los resultados, incluida la pérdida de entrenamiento.

# Definir los Hiperparámetros (argumentos de entrenamiento)

**Notación científica**

*   "1e-3" es una notación abreviada para representar un número en notación
científica. En este caso, "1e-3" significa "1 multiplicado por 10 elevado a la potencia de -3". Simplificando, esto se traduce a "0.001".
*   Entonces, "1e-3" es equivalente a "0.001". Esta notación se utiliza comúnmente en programación y ciencias para representar números pequeños de manera más concisa. En este caso específico, "1e-3" representa el número decimal "0.001".
*   El número "2e-05" es una notación abreviada para representar un número en notación científica. En este caso, "2e-05" significa "2 multiplicado por 10 elevado a la potencia de -5". Simplificando, esto se traduce a "0.00002".
*   Entonces, "2e-05" es equivalente a "0.00002". Esta notación se utiliza comúnmente en la programación y en disciplinas científicas para representar números muy pequeños de manera más concisa.


In [ ]:
# Hiperparámetros
lr = 1e-5             #  Tasa de aprendizaje: tamaño del paso de optimización
batch_size = 8        #  Tamaño del lote: número de ejemplos procesados por paso de optimización
num_epochs = 10       #  Número de épocas: número de veces que el modelo atraviesa los datos de entrenamiento
weight_decay = 0.006  #  Decaimiento de peso para evitar el sobreajuste


Explicación detallada de cada argumento utilizado en la definición de los argumentos de entrenamiento para el fine-tuning del modelo preentrenado de BERT:

1. **output_dir='./results'**: Este argumento especifica el directorio donde se guardarán los resultados del entrenamiento, como los modelos guardados y otros archivos relacionados con el entrenamiento.

2. **num_train_epochs=num_epochs:** Indica el número de épocas de entrenamiento que se realizarán. Cada época representa un pase completo a través de todo el conjunto de datos de entrenamiento.

3. **per_device_train_batch_size=batch_size:** Determina el tamaño del lote de entrenamiento por dispositivo. Especifica cuántos ejemplos se procesarán en paralelo en cada iteración de entrenamiento en cada dispositivo.

4. **per_device_eval_batch_size=bah_size:** Similar al argumento anterior, pero para el tamaño del lote durante la evaluación del modelo.

5. **evaluation_strategy='epoch':** Especifica la estrategia de evaluación, en este caso, se evaluará el modelo al final de cada época.

6. **logging_dir='./logs':** Define el directorio donde se guardarán los registros de entrenamiento, como los registros de pérdida y métricas durante el entrenamiento.

7. **logging_steps=100:** Establece la frecuencia con la que se guardarán los registros durante el entrenamiento. En este caso, se guardarán registros cada 100 pasos de entrenamiento.

8. **save_steps=1000:** Define cada cuántos pasos se guardará el modelo durante el entrenamiento. Aquí, se guardará el modelo cada 1000 pasos.

9. **save_total_limit=3:** Limita el número total de modelos guardados. En este caso, se guardarán hasta 3 modelos.

10. **learning_rate=lr:** Especifica la tasa de aprendizaje inicial para el optimizador. Controla la magnitud de los ajustes realizados a los pesos del modelo durante el entrenamiento.

11. **weight_decay=0.01:** Define el factor de decaimiento de peso para evitar el sobreajuste. Ayuda a regularizar el modelo durante el entrenamiento.

12. **report_to="all":** Indica que se deben reportar todos los resultados, incluida la pérdida de entrenamiento, en los registros y otras salidas.

13. **save_strategy="epoch":** Determina la estrategia de guardado de modelos. En este caso, se guarda el modelo al final de cada época.

14. **load_best_model_at_end=True:** Indica que se debe cargar el mejor modelo al final del entrenamiento. Esto ayuda a asegurar que el modelo final sea el mejor en términos de métricas de evaluación.


In [ ]:
from transformers import TrainingArguments, Trainer

# Definir los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir='./results4',           # Directorio donde se guardarán los resultados del entrenamiento
    num_train_epochs=num_epochs,               # Número de épocas de entrenamiento
    per_device_train_batch_size=batch_size,    # Tamaño del lote de entrenamiento por dispositivo
    per_device_eval_batch_size=batch_size,     # Tamaño del lote de evaluación por dispositivo
    evaluation_strategy='epoch',      # Estrategia de evaluación
    logging_dir='./logs4',            # Directorio donde se guardarán los logs de entrenamiento
    logging_steps=100,                # Cada cuántos pasos se guardarán los logs
    save_steps=1000,                  # Cada cuántos pasos se guardará el modelo
    save_total_limit=3,               # Límite total de modelos guardados
    learning_rate=lr,                 # Tasa de aprendizaje inicial para el optimizador
    weight_decay=weight_decay,        # Decaimiento de peso para evitar sobreajuste
    report_to="all",                  # Reportar todos los resultados, incluida la pérdida de entrenamiento
    save_strategy="epoch",            # Estrategia de guardado del modelo
    load_best_model_at_end=True,      # Cargar el mejor modelo al final del entrenamiento
)

Revisando los `TrainingArguments`, vemos todas las configuraciones relacionadas con el entrenamiento del modelo, desde el directorio donde se guardarán los resultados (output_dir) hasta la tasa de aprendizaje (learning_rate), la estrategia de evaluación (evaluation_strategy), la estrategia de guardado del modelo (save_strategy), entre otros.

Observando el parámetro `optim`, el valor  es `adamw_torch`, lo que indica que se está utilizando el **optimizador AdamW** de PyTorch.

La configuración del optimizador se realiza a través del parámetro `learning_rate`, que indica la tasa de aprendizaje inicial para el optimizador.

La biblioteca transformers suele utilizar optimizadores como **AdamW** (una variante de **Adam** con la corrección de peso de decaimiento) de forma predeterminada, a menos que especifiques un optimizador diferente explícitamente.



La diferencia entre **Adam** y **AdamW** radica en cómo manejan el término de decaimiento de peso (`weight decay`) durante el entrenamiento.

1. **Adam** (*Adaptive Moment Estimation*):

*   Adam es un optimizador popular utilizado en el entrenamiento de redes neuronales. Combina las ventajas de RMSProp y Momentum.
*   En Adam, el término de decaimiento de peso se aplica directamente a los pesos durante la actualización. Sin embargo, esto puede causar problemas de regularización y sobreajuste.

2. **AdamW** (*Adam with Weight Decay*):

*   AdamW es una variante de Adam que aborda el problema de regularización.
*   En AdamW, el término de decaimiento de peso se aplica de manera diferente. Utiliza lo que se llama “*Decoupled Weight Decay Regularization*”. Esto significa que el decaimiento de peso no afecta directamente los pesos durante la actualización, sino que se aplica por separado después de la actualización.
*   La implementación de AdamW asegura que el decaimiento de peso no interfiera con los momentos acumulados, lo que ayuda a prevenir el sobreajuste.



In [ ]:
training_args

TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'gradient_accumulation_kwargs': None},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_steps=None,
evaluation_strategy=epoch,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_v2': False, 'xl

# Congelar capas

Para realizar una afinación por capas en la que algunas capas se mantienen congeladas durante el entrenamiento ("congelación de capas"), se puede utilizar la funcionalidad de PyTorch para congelar los parámetros de las capas que no se quieren ajustar.

In [ ]:
# Congelar las capas que no se quieren afinar
#for param in model.bert.parameters():
#    param.requires_grad = False

Si queremos congelar una capa intermedia específica en lugar de todas las capas BERT preentrenadas, se puede acceder a esa capa específica dentro del modelo y establecer su atributo `requires_grad` en False.

In [ ]:
# Congelar una capa intermedia
#layer_to_freeze = 6  # Por ejemplo, congelar la sexta capa intermedia
#for param in model.bert.encoder.layer[layer_to_freeze].parameters():
#    param.requires_grad = False

# Agregar capas adicionales

La capa `qa_outputs` en el modelo BERT se encarga de generar las respuestas a partir de la representación de los tokens de la pregunta y del contexto.

Cuando agregamos capas adicionales al clasificador de salida del modelo BERT para preguntas y respuestas, estamos extendiendo la arquitectura del modelo al añadir más capas neuronales justo antes de las capas finales que predicen las respuestas. Estas capas adicionales se definen en `new_layers` y luego se agregan al final del clasificador de salida del modelo BERT utilizando el método `add_module()`.

Esto significa que las capas definidas en `new_layers` se integrarán en la arquitectura existente del modelo BERT para ajustarse mejor a la tarea específica de preguntas y respuestas.

Agregar capas adicionales puede permitir al modelo aprender representaciones más complejas y adaptarse mejor a los datos de entrenamiento, lo que puede resultar en un mejor rendimiento en la tarea de preguntas y respuestas.

In [ ]:
# Agregar capas adicionales
num_added_layers = 2                           # Definir el número de capas adicionales a agregar
hidden_size = model.config.hidden_size         # Obtener el tamaño de la capa oculta del modelo BERT
new_layers = torch.nn.Sequential(
    torch.nn.Linear(hidden_size, hidden_size), # Capa lineal
    torch.nn.ReLU(),                           # Función de activación ReLU
    torch.nn.Dropout(0.1),                     # Capa de dropout para regularización
    torch.nn.Linear(hidden_size, hidden_size), # Segunda capa lineal
    torch.nn.ReLU(),                           # Segunda función de activación ReLU
    torch.nn.Dropout(0.1)                      # Segunda capa de dropout
)

# Agregar las capas adicionales al clasificador de salida del modelo BERT
model.qa_outputs.add_module('additional_layers', new_layers)

Observamos como la estructura del modelo BERT se ha ampliado con las nuevas capas

In [ ]:
model

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elem

# Aplicar LoRA al entrenamiento

 Se pueden aplicar técnicas de optimización de la tasa de aprendizaje como **LoRA** (*Linearly decaying on Restart Approach*) o **QLora** (*Quadratic LoRA*) al entrenamiento del modelo BERT que hemos configurado. Estas técnicas pueden ayudar a mejorar el rendimiento del modelo ajustando la tasa de aprendizaje durante el entrenamiento.

## 1. Primero, necesitamos definir una **función de decaimiento** de la tasa de aprendizaje. Para LoRA, la tasa de aprendizaje se ajusta linealmente en cada reinicio del entrenamiento. Se define una función para esto, que tomará como argumento el paso actual, el número total de pasos y la tasa de aprendizaje inicial.

In [ ]:
def learning_rate_decay(step, total_steps, initial_lr):
    num_restarts = 3  # Número de reinicios de entrenamiento
    restart_interval = total_steps // num_restarts
    current_restart = step // restart_interval
    return 1.0 - current_restart * 1.0 / num_restarts


## 2. Luego, necesitamos crear un objeto de programador (**scheduler**) que aplicará esta función de decaimiento de la tasa de aprendizaje al optimizador durante el entrenamiento.

**LoRA (Linearly decaying Learning rate with Restart Algorithm)**

Cuando trabajas con conjuntos de datos pequeños, es importante maximizar la eficiencia del entrenamiento y aprovechar al máximo la información disponible. Una forma de hacerlo es utilizando un optimizador junto con un algoritmo de programación de la tasa de aprendizaje (*learning rate scheduling*) como **LoRA** (*Linearly decaying Learning rate with Restart Algorithm*).

LoRA es un algoritmo de programación de la tasa de aprendizaje que ajusta la tasa de aprendizaje linealmente durante el entrenamiento. Además, LoRA tiene la capacidad de reiniciar la tasa de aprendizaje a un valor inicial después de cierto número de épocas, lo que puede ayudar a escapar de los mínimos locales durante el entrenamiento y a explorar el espacio de búsqueda de manera más efectiva.

El scheduler LoRA se crea utilizando la **función LambdaLR** de PyTorch, que aplica una función lambda para ajustar la tasa de aprendizaje en función del número total de pasos de entrenamiento. Luego, durante el entrenamiento, la tasa de aprendizaje se actualiza en cada iteración de acuerdo con el scheduler.

Se deben ajustar los valores de la tasa de aprendizaje inicial y otros hiperparámetros según sea necesario para el conjunto de datos específico y el modelo.


### Adam (Adaptive Moment Estimation)

 "*Adaptive Moment Estimation*". **Adam** es un algoritmo de optimización ampliamente utilizado en el entrenamiento de redes neuronales, especialmente en problemas de aprendizaje profundo.

El algoritmo Adam combina las ideas de dos enfoques anteriores: el método de momentum, que mantiene un promedio ponderado de las actualizaciones de los pesos anteriores para acelerar la convergencia, y el método de la tasa de aprendizaje adaptativa, que adapta la tasa de aprendizaje de cada peso individualmente según su historia de gradientes.

Las principales características del algoritmo Adam son:

1. Adaptabilidad de la tasa de aprendizaje: Adam calcula una tasa de aprendizaje adaptativa para cada parámetro basándose en estimaciones de primer y segundo orden de los momentos del gradiente.

2. Momentum: Adam utiliza un término de momentum similar al utilizado en el método SGD con momento. Este término de momentum ayuda a suavizar el proceso de optimización y a acelerar la convergencia. El valor por defecto para el momentum en el optimizador Adam es **0.9**. Esto significa que el algoritmo utiliza un coeficiente de 0.9 para acumular inercia en la dirección de los gradientes durante el proceso de optimización.

3. Regularización: Adam incorpora regularización L2 (*Weight Decay*) de forma implícita, lo que ayuda a prevenir el sobreajuste.

En resumen, Adam es un algoritmo de optimización muy efectivo y popular que combina la adaptabilidad de la tasa de aprendizaje con el momentum para mejorar la eficiencia y la estabilidad del entrenamiento de redes neuronales.

**La regularización L2, también conocida como Weight Decay**

Es una técnica utilizada en el entrenamiento de modelos de aprendizaje automático para prevenir el sobreajuste. Aquí están los detalles:

1. La regularización L2 agrega un término de penalización a la función de pérdida durante el entrenamiento del modelo.
2. Este término penaliza los pesos del modelo que son demasiado grandes, lo que ayuda a evitar que los coeficientes se vuelvan excesivamente sensibles a los datos de entrenamiento.
3. Durante el proceso de optimización, la función de pérdida se modifica para incluir un término adicional basado en la norma L2 de los pesos.
4. La norma L2 de un vector de pesos se calcula como la suma de los cuadrados de los valores de los pesos.
5. El término de penalización L2 se agrega a la función de pérdida original, multiplicado por un hiperparámetro llamado factor de regularización.
6. El objetivo es minimizar la función de pérdida total, que ahora incluye tanto la pérdida original como la penalización L2.
7. La regularización L2 ayuda a prevenir el sobreajuste al limitar la magnitud de los pesos del modelo.
8. Al penalizar los pesos grandes, se fomenta la simplicidad del modelo y se evita que se ajuste demasiado a los datos de entrenamiento ruidosos.

En resumen, la **regularización L2** (`Weight Decay`) es una herramienta útil para mejorar la generalización del modelo y garantizar que no se especialice demasiado en los datos de entrenamiento.

**Detalles del optimizador Adam**

1. `amsgrad`: Este parámetro determina si se utiliza la variante AMSGrad de Adam. AMSGrad modifica la regla de actualización para el promedio móvil de los gradientes al cuadrado.
2. `betas`: Estos son los coeficientes para las medias móviles exponenciales del gradiente y su cuadrado. Los valores predeterminados son (0.9, 0.999).
3. `eps`: Es una constante pequeña que se suma al denominador para evitar la división por cero.
4. `initial_lr`: La tasa de aprendizaje inicial.
5. `lr`: La tasa de aprendizaje actual.
6. `weight_decay`: La fuerza de regularización L2 (también conocida como decaimiento de peso). Controla la penalización sobre los pesos del modelo para prevenir el sobreajuste.


### El scheduler LoRA con el optimizador SGD (Stochastic Gradient Descent)

Es posible aplicar el scheduler LoRA con el optimizador SGD (*Stochastic Gradient Descent*) para abordar el problema de un conjunto de datos pequeño. SGD es un algoritmo de optimización ampliamente utilizado en el aprendizaje profundo y es particularmente útil cuando se trabaja con conjuntos de datos pequeños, ya que puede ser más efectivo para explorar el espacio de búsqueda de manera más exhaustiva.

En nuestro caso, se utiliza el optimizador SGD con un momentum de 0.95 y una tasa de aprendizaje inicial de 0.01 (`lr = 1e-5)`. Luego, se crea el scheduler LoRA utilizando la función LambdaLR de PyTorch, que ajusta la tasa de aprendizaje en función del número total de pasos de entrenamiento. Durante el entrenamiento, la tasa de aprendizaje se actualiza en cada iteración según el scheduler.

Se debe tener en cuenta que los valores de la tasa de aprendizaje inicial, el momentum y otros hiperparámetros pueden variar según el conjunto de datos y el modelo específico, por lo que es importante ajustarlos según sea necesario.



El parámetro **momentum** en el optimizador SGD controla la aceleración de la optimización. Es una técnica que ayuda a mejorar la convergencia y a reducir el ruido en los datos. Básicamente, el momentum acumula una fracción del gradiente pasado para acelerar el descenso del gradiente en la dirección dominante y disminuir las oscilaciones en direcciones menos importantes.

Si se aplica la función pero no se obtienen resultados satisfactorios, aquí hay algunas opciones que a considerar:

1. Tasa de aprendizaje: Probar con diferentes tasas de aprendizaje. A veces, una tasa de aprendizaje más baja puede ayudar a la convergencia, especialmente en conjuntos de datos pequeños.

2. Número de épocas: Aumentar el número de épocas de entrenamiento si el modelo aún no ha convergido. Esto puede ayudar a que el modelo aprenda patrones más complejos en los datos.

3. Regularización: Ajustar el parámetro de regularización (`weight_decay en Adam, o weight_decay en SGD`) para controlar el sobreajuste y mejorar el rendimiento del modelo en los datos de evaluación.

4. Explorar otros optimizadores: Además de SGD y Adam, hay otros optimizadores como RMSprop, Adagrad, etc. Podemos probar otros optimizadores para ver si mejoran el rendimiento del modelo.

5. Ajustar el momentum: El valor de momentum puede influir en la convergencia del modelo. Experimentar con diferentes valores para el momentum y observar cómo afecta el rendimiento del modelo.



El parámetro **momentum** en el optimizador SGD controla la velocidad y dirección de las actualizaciones de los pesos durante el entrenamiento. Es una medida de la inercia del proceso de optimización, es decir, cuánto se basa la dirección de la actualización en las actualizaciones anteriores.

Para un conjunto de datos de alrededor de 500 preguntas y respuestas, el valor óptimo para el parámetro momentum puede variar dependiendo de la naturaleza específica de los datos y del modelo. Sin embargo, típicamente se suele utilizar un valor entre 0.9 y 0.99. Estos valores son comunes en la práctica y suelen funcionar bien en muchas situaciones.

Aquí hay algunas pautas generales:

1. Valor Mínimo: El valor mínimo que se suele utilizar es 0, ya que significa que no hay inercia en las actualizaciones de los pesos y cada actualización se basa únicamente en el gradiente actual.

2. Valor Máximo: El valor máximo suele ser 1. Valores más altos pueden llevar a una mayor inercia, lo que puede ayudar a evitar que el optimizador se atasque en óptimos locales poco profundos.

Dicho esto, es recomendable realizar pruebas con diferentes valores de momentum en un rango de 0.1 a 0.99 para determinar cuál funciona mejor para tu conjunto de datos y modelo específicos. Vamos a comenzar con un valor de 0.9 y ajustarlo según los resultados de la validación. Despues de varios entrenamientos, lo fijamos en **0.95**.

**Detalles del optimizador SGD**

1. `dampening`: Este parámetro no se utiliza en la versión estándar de SGD. Es relevante en otros optimizadores, pero en SGD se mantiene en 0.
2. `differentiable`: Si está configurado en False, significa que el optimizador no es diferenciable. Esto podría ser útil en casos específicos donde se requiere un enfoque no diferenciable.
3. `initial_lr`: La tasa de aprendizaje inicial. Es el valor que se utiliza al principio del entrenamiento antes de que se realicen ajustes.
4. `lr`: La tasa de aprendizaje actual. Durante el entrenamiento, esta tasa se ajusta según el rendimiento del modelo.
5. `maximize`: Si está configurado en False, el optimizador minimiza la función de pérdida. Si se establece en True, maximiza una función objetivo (por ejemplo, en problemas de maximización).
6. `momentum`: El valor de momentum utilizado en SGD. El momentum es una técnica que acelera la convergencia al acumular inercia en la dirección de los gradientes.
7. `nesterov`: Si está configurado en False, se utiliza el método clásico de Nesterov para calcular el momentum. Si se establece en True, se utiliza la variante de Nesterov.
8. `weight_decay`: La fuerza de regularización L2 (Weight Decay). Controla la penalización sobre los pesos del modelo para prevenir el sobreajuste.


## 3. Finalmente, se pasa el **scheduler** junto con el optimizador al Trainer al crearlo.

Inicializa el Trainer con los argumentos y el scheduler

trainer = Trainer(

    model=model,    
    args=training_args,        
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],  
    tokenizer=tokenizer,             
    data_collator=data_collator,          
    optimizers=(optimizer, scheduler)
)

## DefaultDataCollator

In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

**DefaultDataCollator**

El DefaultDataCollator es una parte esencial del proceso de entrenamiento de modelos de lenguaje natural (NLP). Es un recopilador de datos simple incluido en la biblioteca Transformers. Realiza un procesamiento básico en varios formatos de datos utilizados comúnmente para entrenar modelos de transformadores. Su función principal es preparar los datos de entrada de manera adecuada antes de pasarlos al modelo para el entrenamiento o la evaluación. Algunas de las tareas típicas que realiza un data collator incluyen:

1. Formato de datos:

Asume que tus datos están en forma de diccionarios, donde las claves representan tipos de datos (por ejemplo, "input_ids", "labels") y los valores son tensores que contienen los datos correspondientes.

2. Manejo especial de etiquetas:

Realiza un manejo específico para dos claves potenciales en los diccionarios:

*   `label`: Si está presente, asume que esta clave contiene un solo valor (entero o flotante) que representa la etiqueta de la muestra.
*   `label_ids`: Si está presente, asume que esta clave contiene una lista de valores que representan las etiquetas de la muestra (utilizada en tareas como la clasificación de secuencias).

label: Si está presente, asume que esta clave contiene un solo valor (entero o flotante) que representa la etiqueta de la muestra.
label_ids: Si está presente, asume que esta clave contiene una lista de valores que representan las etiquetas de la muestra (utilizada en tareas como la clasificación de secuencias).

3. Relleno de tokens: Asegura que todas las secuencias de entrada tengan la misma longitud agregando tokens de relleno (generalmente tokens especiales como [PAD]) según sea necesario para igualar la longitud máxima en un lote.

4. Agrupación de ejemplos en lotes: Toma una lista de ejemplos individuales y los organiza en lotes de tamaño uniforme para el procesamiento eficiente en paralelo durante el entrenamiento.

El DefaultDataCollator proporciona una implementación predeterminada que maneja estas tareas comunes de manera eficiente. Sin embargo, en casos más específicos o personalizados, es posible que desees crear tu propio data collator para adaptarse a las necesidades de tu aplicación.

En resumen:

Se importa y crea una instancia de la clase DefaultDataCollator, un recopilador de datos básico adecuado para escenarios de entrenamiento simples con datos formateados en diccionarios en Transformers.


# Inicialización de Trainer con los argumentos y optimizadores y Entrenamiento

El objeto Trainer es inicializado con varios parámetros, incluyendo el modelo preentrenado, los argumentos de entrenamiento, los conjuntos de datos tokenizados para entrenamiento y evaluación, el tokenizer, el data_collator (para el relleno dinámico de secuencias), y los optimizadores (que incluyen el optimizador y el scheduler).

El scheduler que esta definido, se pasará como parte de la tupla de optimizadores al objeto Trainer, lo que significa que se utilizará durante el entrenamiento para ajustar dinámicamente la tasa de aprendizaje. En cada iteración de entrenamiento, el scheduler actualizará la tasa de aprendizaje según el esquema especificado (en este caso, LoRA).

Entonces, cuando se llama al método trainer.train(), el entrenamiento comienza utilizando el modelo, los datos de entrenamiento y evaluación, y los optimizadores con el scheduler incluido. Durante el entrenamiento, el scheduler se encargará de actualizar la tasa de aprendizaje según el número total de pasos de entrenamiento, siguiendo el esquema de LoRA que se ha definido. Esto permite una adaptación dinámica de la tasa de aprendizaje durante el proceso de entrenamiento, lo que puede ser beneficioso para obtener un mejor rendimiento, especialmente cuando se trabaja con conjuntos de datos pequeños.

 **Padding** en la función de tokenización y el **DefaultDataCollator** en el Trainer.

Cuando se crea el Trainer para entrenar el modelo BERT con fine-tuning, puedes proporcionar un DataCollator como el DefaultDataCollator que se encargará de manejar el procesamiento final de los datos antes de pasarlos al modelo durante el entrenamiento. La función principal del DataCollator es realizar el relleno de tokens y la agrupación de ejemplos en lotes. En este caso, se ha utilizado:

*   **padding="max_length"** en la tokenización, el DefaultDataCollator se encargará de agregar tokens de padding según sea necesario para igualar la longitud máxima en un lote.
*   El **padding** (padding="max_length") especificado durante la tokenización se reflejará en el proceso de relleno de tokens realizado por el **DefaultDataCollator** para asegurar que todas las secuencias en un lote tengan la misma longitud. Esto es esencial para el entrenamiento eficiente del modelo.


In [ ]:
# Inicializa el Trainer con los argumentos y el scheduler
trainer = Trainer(
    model=model,                                    # Modelo preentrenado que deseas afinar
    args=training_args,                             # Argumentos de entrenamiento que has definido
    train_dataset=tokenized_datasets["train"],      # Conjunto de datos de entrenamiento tokenizado
    eval_dataset=tokenized_datasets["validation"],  # Conjunto de datos de evaluación tokenizado
    tokenizer=tokenizer,                            # Tokenizador
    data_collator=data_collator,                    # Padding: relleno de secuencia dinámica
)

Durante la ejecución del entrenador (trainer), las tres columnas representan la siguiente información:

1. `Epoch`: Esta columna indica el número de época actual durante el entrenamiento del modelo. Una época se refiere a una pasada completa a través de todo el conjunto de datos de entrenamiento.

2. `Training Loss` (Pérdida de entrenamiento): Esta columna muestra la pérdida calculada durante el entrenamiento en cada época. La pérdida de entrenamiento es una medida de qué tan bien se está ajustando el modelo a los datos de entrenamiento. Se calcula comparando las predicciones del modelo con las etiquetas verdaderas del conjunto de datos de entrenamiento.

3. `Validation Loss` (Pérdida de validación): Esta columna muestra la pérdida calculada durante la evaluación en el conjunto de datos de validación en cada época. La pérdida de validación es una medida de qué tan bien generaliza el modelo a datos que no ha visto durante el entrenamiento. Se calcula de manera similar a la pérdida de entrenamiento, pero utilizando el conjunto de datos de validación en lugar del conjunto de datos de entrenamiento.

El objetivo principal del entrenamiento es minimizar tanto la pérdida de entrenamiento como la de validación. **Una pérdida de entrenamiento baja indica que el modelo está ajustando bien los datos de entrenamiento, mientras que una pérdida de validación baja indica que el modelo generaliza bien a datos no vistos**. Sin embargo, es importante vigilar ambas pérdidas para detectar signos de sobreajuste (cuando la pérdida de entrenamiento sigue disminuyendo pero la pérdida de validación comienza a aumentar).

In [ ]:
# Entrenar el modelo
%%time
trainer.train()


Epoch,Training Loss,Validation Loss
1,No log,1.174107
2,1.387200,0.723761
3,1.387200,0.637797
4,0.271700,0.696578
5,0.271700,0.718242
6,0.091100,0.698468
7,0.091100,0.731071
8,0.032600,0.807296
9,0.032600,0.770403
10,0.017000,0.766648


CPU times: user 6min 54s, sys: 22.1 s, total: 7min 16s
Wall time: 7min 57s


TrainOutput(global_step=510, training_loss=0.35299577696942813, metrics={'train_runtime': 476.6901, 'train_samples_per_second': 8.475, 'train_steps_per_second': 1.07, 'total_flos': 1070298430095360.0, 'train_loss': 0.35299577696942813, 'epoch': 10.0})

# Un valor de pérdida de entrenamiento de aproximadamente 0.5

Un valor de pérdida de entrenamiento de 0.5, por ejemplo,  es relativamente bajo, lo que indica que el modelo está haciendo un buen trabajo en la tarea de entrenamiento. Sin embargo, la interpretación de la pérdida de entrenamiento depende en gran medida del contexto específico del problema y del rango típico de valores de pérdida para el problema en particular.

Aquí hay algunas consideraciones para evaluar si este valor de pérdida es bueno o no:

1. Comparación con la pérdida de validación: Es importante comparar la pérdida de entrenamiento con la pérdida de validación. Si la pérdida de entrenamiento es baja pero la pérdida de validación es alta, podría indicar que el modelo está sobreajustando los datos de entrenamiento y no generaliza bien a datos nuevos.

2. Comparación con iteraciones anteriores: Si tienes registros de pérdida de entrenamiento de iteraciones anteriores, puedes comparar el valor actual con los valores anteriores para ver si ha habido una mejora o empeoramiento.

3. Contexto del problema: Algunos problemas pueden tener pérdidas de entrenamiento más bajas que otros debido a la complejidad de la tarea. Es importante considerar el contexto del problema y si este valor de pérdida es razonable dadas las circunstancias.

4. Objetivo del entrenamiento: Dependiendo de tus objetivos de entrenamiento, un valor de pérdida de entrenamiento determinado puede ser considerado como aceptable o no. Si el objetivo es simplemente entrenar un modelo para una tarea específica, un valor de pérdida de entrenamiento de 0.5 puede ser suficiente. Sin embargo, si estás buscando un rendimiento óptimo en una métrica específica, es posible que necesites optimizar aún más el modelo.

En resumen, un valor de pérdida de entrenamiento de 0.5 parece ser bastante bajo, pero es importante considerar otros factores, como la pérdida de validación y el contexto del problema, para evaluar si este valor es satisfactorio para las necesidades específicas.

# Método train()

El **método train()** se encargará de iterar sobre el conjunto de datos de entrenamiento y ajustar el modelo a los datos.

**Análisis del resultado** de trainer.train():

El resultado que obtienes al ejecutar import torch, trainer.train() en PyTorch representa **el rendimiento de una época de entrenamiento para tu modelo**. Veamos qué significa cada elemento:

1. **TrainOutput:** Esta es una estructura de datos personalizada que contiene información sobre el entrenamiento.
2. **global_step ()**: Indica el número de iteración o paso global actual en el proceso de entrenamiento. Comienza en 0 y aumenta después de procesar cada lote.
3. **training_loss ():** Este es el valor promedio de la pérdida calculado en todos los lotes de la época actual. Un valor de pérdida más bajo generalmente indica un mejor rendimiento del modelo.
4. **metrics (diccionario)**: Este diccionario contiene varias métricas registradas durante el entrenamiento:
 *   `train_runtime ()`: Tiempo que tardó en entrenar el modelo durante toda la época (en segundos).
 *   `train_samples_per_second ()`: Promedio de muestras de entrenamiento procesadas por segundo durante la época.
 *   `train_steps_per_second ()`: Promedio de pasos de entrenamiento (lotes) procesados por segundo durante la época.
 *   `total_flos ()`: Número total de FLOPs (operaciones de punto flotante) realizadas durante el entrenamiento en la época. Esta métrica puede ser útil para comprender el uso computacional.
 *   `train_loss ()`: Duplicado del valor de la pérdida de entrenamiento mencionado anteriormente.
 *   `epoch ()`: Especifica el número de época actual.

En resumen, este resultado proporciona información valiosa sobre el progreso del entrenamiento de tu modelo después de completar todas las épocas. Se puede usar esta información para:

*   Monitorizar la pérdida: Rastrear cómo la pérdida de entrenamiento disminuye con las épocas, lo que indica una posible convergencia.
*   Evaluar la velocidad de entrenamiento: Analizar `train_runtime, train_samples_per_second y train_steps_per_second` para comprender la eficiencia del entrenamiento.
*   Optimizar el uso de recursos: Evaluar `total_flos` para comprender los requisitos computacionales del entrenamiento.

Se recomendable monitorizar estas métricas a lo largo de varias épocas para obtener una imagen más clara del proceso de entrenamiento.

Cuando durante el entrenamiento de un modelo se muestra "`No log`" en el "Training Loss", significa que no se ha registrado ningún valor para la pérdida durante ese paso de entrenamiento específico. Esto podría ocurrir por varias razones:

1. **Problemas durante el entrenamiento**: Puede haber habido un error o un problema durante el proceso de entrenamiento que impidió que se registrara la pérdida.

2. **Configuración incorrecta**: La configuración del entrenamiento o la forma en que se está calculando la pérdida podrían ser incorrectas, lo que podría resultar en valores inesperados o la falta de registro de la pérdida.

3. **Falta de retroalimentación**: Es posible que no se esté proporcionando la retroalimentación adecuada al modelo durante el entrenamiento, lo que podría hacer que no se calcule o registre la pérdida.

En resumen, "No log" en el "Training Loss" indica que no se ha registrado ningún valor de pérdida durante ese paso de entrenamiento, lo cual es anormal y requiere una investigación adicional para comprender y corregir el problema.

# Aumentar la retroalimentación

Para aumentar la retroalimentación y asegurar de que se registra algún valor de pérdida durante el proceso de entrenamiento, se pueden considerar las siguientes opciones:

1. Verificar la configuración del optimizador: Asegúrarse de que se esta utilizando un optimizador adecuado para la tarea de entrenamiento. Experimentar con diferentes optimizadores y ajustar los hiperparámetros correspondientes, como la tasa de aprendizaje y el decaimiento de peso, para mejorar la convergencia del modelo.

2. Revisar la función de pérdida: Verificar que se esta utilizando la función de pérdida correcta para tu tarea específica. Dependiendo del tipo de problema que se aborda (clasificación, regresión, etc.), es importante elegir una función de pérdida adecuada que refleje la naturaleza de tus datos y tus objetivos de entrenamiento.

3. Aumentar el tamaño del lote: A veces, el uso de lotes más grandes durante el entrenamiento puede proporcionar una retroalimentación más estable y significativa para el modelo, lo que puede ayudar a que se registren valores de pérdida durante el proceso de entrenamiento.
Para aumentar el tamaño del lote, simplemente se puede incrementar el valor de `batch_size` en los argumentos de entrenamiento. Por ejemplo, si se desea duplicar el tamaño del lote, se podría establecer batch_size en el doble del valor actual. Sin embargo, hay que tener en cuenta que aumentar demasiado el tamaño del lote puede consumir más memoria y recursos computacionales. Es importante encontrar un equilibrio entre un tamaño de lote más grande y los recursos disponibles.

4. Ajustar la arquitectura del modelo: Si utilizamos una arquitectura de red neuronal compleja, hay que considera simplificarla o ajustarla para reducir la posibilidad de problemas de convergencia durante el entrenamiento.

5. Verificar la calidad de los datos de entrada: Asegúrar de que los datos de entrenamiento estén limpios y bien preparados. Los datos de baja calidad o mal formateados pueden dificultar el entrenamiento del modelo y dificultar la convergencia.

6. Agregar más datos de entrenamiento: Si es posible, aumentar el tamaño de tu conjunto de datos de entrenamiento. Más datos de entrenamiento pueden proporcionar una retroalimentación más rica y variada para el modelo, lo que puede ayudar a mejorar la calidad del entrenamiento y la convergencia del modelo.

7. Explorar técnicas de regularización: Considerar el uso de técnicas de regularización, como la deserción (*dropout*) o la penalización L2, para ayudar a controlar el sobreajuste y mejorar la estabilidad del entrenamiento.

Al experimentar con estas opciones y realizar ajustes en el proceso de entrenamiento, se debería poder aumentar la retroalimentación y visualizar de que se registren valores de pérdida durante el entrenamiento del modelo.

# Optimizador

Dado que estamos trabajando con un conjunto de datos relativamente pequeño (500), hay algunas modificaciones que se pueden considerar en el código del optimizador para mejorar el entrenamiento de tu modelo. Algunas opciones:

1. Tasa de aprendizaje inicial (`learning rate`): Dado que tenemos un conjunto de datos pequeño, es posible ajustar la tasa de aprendizaje inicial para controlar la velocidad a la que el modelo aprende de los datos. Una tasa de aprendizaje más baja puede ayudar a evitar un sobreajuste rápido en conjuntos de datos pequeños.

2. Decaimiento de peso (`weight decay`): El decaimiento de peso se utiliza para penalizar los pesos grandes en la función de pérdida durante el entrenamiento, lo que puede ayudar a prevenir el sobreajuste. Sin embargo, en conjuntos de datos pequeños, un decaimiento de peso demasiado alto podría limitar la capacidad del modelo para aprender de los datos. Hay que considerar reducir el valor de decaimiento de peso o incluso omitirlo inicialmente para ver cómo afecta al entrenamiento.

3. Explorar otras optimizaciones: Además de **Adam**, se pueden explorar otras opciones de optimización que podrían funcionar mejor para conjuntos de datos pequeños. Por ejemplo, **SGD** (*Descenso de Gradiente Estocástico*) con impulso (`momentum`) o adaptación del tamaño del paso (`learning rate scheduling`) podría ser más efectivo en este escenario.

Es importante realizar experimentos y ajustes iterativos en la configuración del optimizador para encontrar lo que funciona mejor para tu conjunto de datos y tu modelo específico.


# Guardar el modelo entrenado en disco

La instrucción `trainer.save_model()` se utiliza en el contexto del paquete `transformers` de Hugging Face en Python y sirve para guardar el modelo entrenado en disco. Esta función es parte del objeto `Trainer` que se utiliza para entrenar modelos de `transformers`.

Cuando entrenas un modelo con el `Trainer`, después de que el entrenamiento ha concluido y estás satisfecho con el rendimiento del modelo, puedes llamar a `trainer.save_model()` para guardar todos los pesos y configuraciones del modelo en un directorio específico.

Cuando utilizas trainer.save_model(), el modelo entrenado se guarda en un directorio específico en tu sistema de archivos. Por defecto, este directorio se llama output y se crea en el directorio de trabajo actual. Dentro de este directorio output, se creará una carpeta adicional con el nombre del modelo, y dentro de esta carpeta se guardarán los archivos relacionados con el modelo, como los pesos del modelo y los archivos de configuración.

Si deseas especificar una ubicación diferente para guardar el modelo, puedes pasar la ruta de destino como argumento a la función `trainer.save_model()`:

En caso de no existir la carpeta MODELOS, se crea automaticamente y los 7 archivos del modelo se guardan.

trainer.save_model('/content/drive/MyDrive/MODELOS/OptimizadorAdamW')

Se crean estos archivos:

1. config.json

2. model.safetensors

3. special_tokens_map.json

4. tokenizer.json

5. tokenizer_config.json

6. training_args.bin

7. vocab.txt

**Explicación de cada archivo guardado por trainer.save_model():**

1. **config.json**: Este archivo contiene la configuración del modelo, incluyendo los hiperparámetros, la arquitectura y cualquier otra configuración necesaria para recrear el modelo.

2. **model.safetensors**: Este archivo contiene los pesos del modelo entrenado. Es esencial para cargar el modelo entrenado en el futuro y realizar inferencias o continuar el entrenamiento desde donde lo dejaste.

3. **special_tokens_map.json**: Este archivo mapea los tokens especiales utilizados por el tokenizador a sus identificadores correspondientes. Estos tokens especiales pueden ser tokens de inicio, tokens de fin, tokens de relleno, etc.

4. **tokenizer.json**: Este archivo contiene la información necesaria para recrear el tokenizador utilizado durante el entrenamiento del modelo. Incluye el vocabulario, las configuraciones de tokenización y cualquier otra información relevante.

5. **tokenizer_config.json**: Similar al archivo tokenizer.json, este archivo contiene la configuración del tokenizador.

6. **training_args.bin**: Este archivo almacena los argumentos de entrenamiento utilizados durante el entrenamiento del modelo. Esto incluye información sobre la configuración del entrenamiento, como la tasa de aprendizaje, el tamaño del lote, el número de épocas, etc.

7. **vocab.txt**: Este archivo contiene el vocabulario utilizado por el tokenizador. Esencialmente, mapea los tokens a sus identificadores correspondientes y viceversa. Este archivo se utiliza durante la tokenización del texto de entrada durante la inferencia o el entrenamiento.

In [ ]:
trainer.save_model('/content/drive/MyDrive/MODELOS/OptimizadorAdamW')

Ver la carpeta content
('/content/drive/MyDrive/MODELOS')

**Análisis y conclusiones sobre la calidad del modelo:**

Basándonos en los resultados obtenidos en el entrenamiento:

*   Pérdida de entrenamiento (): La pérdida de entrenamiento parece ser relativamente baja, lo que indica un buen rendimiento del modelo en esta etapa inicial.
*   Velocidad de entrenamiento: Los valores de `train_samples_per_second y train_steps_per_second` sugieren una velocidad de entrenamiento razonable

Aspectos a considerar:

*   Número de épocas de entrenamiento: Variar diferentes número de épocas.
*   Conjunto de datos: Tamaño.
*   Tarea: Tipo de tarea (generación de texto).
*   Métricas de evaluación: Métricas relevantes para la tarea específica (precisión, F1 score, etc.).

Siguientes procesos:

*   Continuar el entrenamiento: Es recomendable continuar el entrenamiento durante más épocas para observar cómo evoluciona la pérdida y otras métricas relevantes.
*   Evaluar el rendimiento: Una vez finalizado el entrenamiento, se debe evaluar el rendimiento del modelo en el conjunto de datos de validación independiente para obtener una medida precisa de su calidad.
*   Comparar con otros modelos: Si es posible, comparar el rendimiento del modelo con otros modelos existentes para la misma tarea.

En resumen:

*   Los resultados iniciales parecen indicar un modelo con un buen potencial. Sin embargo, se necesita más información y evaluación para determinar la calidad final del modelo en el contexto de la tarea específica.
*   La calidad de un modelo depende en gran medida de la calidad del conjunto de datos, la tarea a realizar y las métricas de evaluación utilizadas.
*   Es importante comparar el modelo con otros modelos existentes para tener una referencia.
*   El entrenamiento y la evaluación de modelos pueden ser procesos iterativos que requieren ajustes y refinamientos.


# Prueba con una instancia del conjunto de entrenamiento

Usaremos la instancia con índice 7 en el conjunto de datos como ejemplo.

In [ ]:
instance = data['train'][12]
context = instance['context']
question = instance['question']

In [ ]:
context

'Sí, hemos puesto en marcha un nuevo servicio para comunicarte de manera inmediata las incidencias que pudieran producirse en la red de distribución, bien por obras de mejora o por situaciones sobrevenidas, y la duración estimada de la suspensión temporal del suministro. Este servicio se ha desarrollado para que puedas adoptar las medidas necesarias para reducir el impacto que estas intervenciones pudieran ocasionar en tu actividad diaria. Ponte en contacto con nosotros a través del teléfono gratuito 900 365 365, o date de alta desde tu perfil (para ello deberás estar registrado).'

In [ ]:
question

'¿Hay alguna app para saber si no va a haber agua?'

Encuentrar la respuesta dada y su posición inicial en el contexto:

In [ ]:
instance['answers']

{'answer_start': [4],
 'text': ['Hemos puesto en marcha un nuevo servicio para comunicarte de manera inmediata las incidencias que pudieran producirse en la red de distribución']}

In [ ]:
given_answer = instance['answers']['text'][0]  # Suponiendo que la primera respuesta sea la correcta
given_answer_start = instance['answers']['answer_start'][0]
given_answer, given_answer_start

('Hemos puesto en marcha un nuevo servicio para comunicarte de manera inmediata las incidencias que pudieran producirse en la red de distribución',
 4)

# Tokenizar los datos de un ejemplo con el modelo

In [ ]:
# Tokenizar los datos con el tokenizer1
inputs = tokenizer(question, context, return_tensors='pt', max_length=512, truncation=True)

# Aplicar el modelo BERT a los datos de un ejemplo


##  **`torch` es una biblioteca de Python**

`torch` es una biblioteca de Python que se utiliza principalmente para cálculos numéricos y operaciones con tensores. Breve explicación:

1. **Tensores**:

*   En el contexto de aprendizaje profundo (deep learning), los tensores son estructuras de datos similares a matrices multidimensionales.
*   PyTorch proporciona una implementación eficiente de tensores, lo que permite realizar cálculos en CPU o GPU.
*   Los tensores son fundamentales para representar datos, parámetros de modelos y gradientes durante el entrenamiento.

2. **Funcionalidades de PyTorch**:

*   Autograd: PyTorch utiliza un sistema de seguimiento automático de gradientes llamado autograd. Esto permite calcular automáticamente gradientes para funciones definidas por el usuario.
*   Redes neuronales: PyTorch facilita la creación y entrenamiento de redes neuronales. Puedes definir tus propias arquitecturas de red o utilizar modelos preentrenados.
*   Optimización: PyTorch proporciona optimizadores como SGD, Adam, etc., para ajustar los parámetros de los modelos durante el entrenamiento.
*   Manipulación de datos: Puedes cargar datos, transformarlos y prepararlos para el entrenamiento utilizando las herramientas de manipulación de datos de PyTorch.

3. **Compatibilidad con GPU**:

*   PyTorch está diseñado para aprovechar la potencia de las GPUs.
*   Al mover tensores y modelos a la GPU (usando `to(device)`), puedes acelerar significativamente los cálculos.

En resumen, `torch` te permite acceder a todas estas funcionalidades y herramientas para trabajar con tensores, construir modelos de aprendizaje profundo y realizar cálculos numéricos. Es una biblioteca esencial en el campo del aprendizaje profundo.

In [ ]:
import torch

##  **Selección del Dispositivo (CPU o GPU):**

Crea un objeto `device` que representa la unidad de procesamiento en la que se ejecutará el modelo.

Si hay una GPU disponible, se selecciona “cuda” (para aprovechar la aceleración de GPU). De lo contrario, se utiliza “cpu”.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

##  **Mover el Modelo al Dispositivo Seleccionado:**

`model.to(device)` mueve el modelo `model` (un modelo preentrenado de BERT) al dispositivo especificado.

Esto es necesario para asegurarse de que los cálculos se realicen en la GPU (si está disponible) o en la CPU.

In [ ]:
model.to(device)

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elem

##  **Preparación de Entradas para el Modelo:**

La línea `inputs = {k: v.to(device) for k, v in inputs.items()}` mueve todos los tensores en el diccionario inputs al dispositivo seleccionado.

El diccionario `inputs` contiene los datos tokenizados necesarios para la inferencia del modelo.

In [ ]:
inputs = {k: v.to(device) for k, v in inputs.items()}

In [ ]:
inputs

{'input_ids': tensor([[    4,  1063,  1311,  2404, 26542,  1097,  2272,  1096,  1054,  1453,
           1012,  1887,  2326,  1059,     5,  1212,  1019,  2392,  2982,  1035,
           4587,  1044,  1827,  2809,  1097, 16403,  1043,  1009,  2160, 11117,
           1085, 14889, 30958,  1041, 10037, 20566,  1035,  1032,  2946,  1009,
           5370,  1019,  1214,  1076,  3915,  1009,  6076,  1068,  1076,  6290,
          27612,  1017,  1416,  1019,  1040,  1032,  6479, 12128,  1009,  1032,
          10928,  8040,  1081,  8130,  1008,  1277,  2809,  1057,  1075,  8204,
           1097,  1041,  6869,  5181,  1085,  2235,  6068,  1097,  4952,  1039,
           6501,  1041,  1932,  9763, 10037, 10239,  1020,  1035,  1203,  3898,
          15359,  1008,  9004,  1035,  4192,  1048,  1905,  1012,  2589,  1081,
           3757, 14854, 12269,  1306,  1000, 31001,  1306,  1000, 31001,  1019,
           1068,  9947,  1009,  3531,  1540,  1203,  9471,  1147,  1097,  2601,
          27834,  1553, 107

##  **Inferencia del Modelo BERT:**

La línea `output = model(**inputs)` realiza la inferencia en el modelo preentrenado de BERT.

El modelo procesa las entradas (pregunta y contexto) y produce dos tensores de salida: `start_logits` y `end_logits`.

Estos tensores contienen las puntuaciones de inicio y fin para la respuesta dentro del contexto.

In [ ]:
# Obtener la salida del modelo
with torch.no_grad():
    output = model(**inputs)

In [ ]:
output

QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[ -6.8356,  -8.8600,  -8.8069,  -8.6484,  -8.8668,  -8.5986,  -8.6861,
          -8.8857,  -9.0867,  -9.1924,  -9.2662,  -9.3141,  -9.9464,  -9.2571,
          -9.0001,  -3.1679,  -4.1773,   4.5418,  -3.9870,  -7.1511,  -7.2012,
          -3.3223,  -3.4989,  -5.5718,  -2.3634,  -4.5314,  -6.0305,  -7.6816,
          -7.7463,  -8.1801,  -7.7552,  -7.8774,  -9.1436,  -9.4272,  -8.7859,
          -9.3019,  -9.5298,  -8.9870,  -8.6762,  -8.7731,  -4.3661,  -7.0233,
           1.4421,  -6.5765,  -6.3976,  -8.8510,  -8.9471,  -9.0217,  -8.8591,
          -8.8715,  -9.3382,  -9.7612,  -7.7736,  -8.8111,  -7.0689,  -3.5694,
          -7.1810,  -8.6847,  -9.0258,  -8.3808,  -8.3396,  -9.4874,  -8.8518,
          -8.5527,  -8.8610,  -5.6966,  -7.5567,  -8.7831,  -8.3336,  -9.2509,
          -8.5028,  -8.5687,  -7.8599,  -8.5034,  -9.1545,  -9.0020,  -9.2837,
          -9.3677,  -9.1293,  -9.4875,  -9.6329,  -9.6608,  -9.1338,  -9.1063,

## **Obtención de la Respuesta Predicha:**

`start_idx` y `end_idx` son los índices con las puntuaciones máximas para el inicio y el final de la respuesta, respectivamente.

Se extraen los tokens correspondientes a esa ventana de índices.

Luego, se convierten los tokens en una cadena de texto (respuesta predicha).

In [ ]:
# Obtener la respuesta prevista
start_idx = torch.argmax(output.start_logits)
end_idx = torch.argmax(output.end_logits)

predicted_answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][start_idx:end_idx + 1]))

## **Información Adicional:**

`start_idx.item()` y `end_idx.item()` devuelven los valores enteros de los índices.

In [ ]:
predicted_answer, start_idx, end_idx, start_idx.item(), end_idx.item()

('hemos puesto en marcha un nuevo servicio para comunicarte de manera inmediata las incidencias que pudieran producirse en la red de distribución',
 tensor(17, device='cuda:0'),
 tensor(40, device='cuda:0'),
 17,
 40)

En resumen, este código se utiliza para aplicar un modelo preentrenado de BERT a una instancia de datos (pregunta y contexto), obtener las puntuaciones de inicio y fin para la respuesta y convertir esos índices en una respuesta predicha.

# Evaluar el resultado de los datos de ejemplo con el modelo inicial

In [ ]:
predicted_answer.lower()

'hemos puesto en marcha un nuevo servicio para comunicarte de manera inmediata las incidencias que pudieran producirse en la red de distribución'

In [ ]:
start_idx

tensor(17, device='cuda:0')

In [ ]:
end_idx

tensor(40, device='cuda:0')

In [ ]:
given_answer.lower()

'hemos puesto en marcha un nuevo servicio para comunicarte de manera inmediata las incidencias que pudieran producirse en la red de distribución'

In [ ]:
predicted_answer.lower()

'hemos puesto en marcha un nuevo servicio para comunicarte de manera inmediata las incidencias que pudieran producirse en la red de distribución'

In [ ]:
given_answer.lower()

'hemos puesto en marcha un nuevo servicio para comunicarte de manera inmediata las incidencias que pudieran producirse en la red de distribución'

In [ ]:
correct = (predicted_answer.lower() == given_answer.lower())
evaluation = 'Correcto' if correct else f'Incorrecto (Predicción: {predicted_answer}, Solución: {given_answer})'

print(evaluation)

Correcto


**Observación:** Si la evaluarion es correcta, el accuracy será 1. Pero si la evaluación difiere en algun caracter, el accuracy será 0, pero es posible que la respuesta tenga validez. Por ejemplo:

1. Predicción: si, hemos puesto en marcha un nuevo servicio para comunicarte de manera inmediata las incidencias que pudieran producirse en la red de distribucion

2. Solución: Hemos puesto en marcha un nuevo servicio para comunicarte de manera inmediata las incidencias que pudieran producirse en la red de distribución

Entonces, en la posterior evaluacion del dataset, basandonos en metricas como el accuracy, debemos tener en cuenta que el accuracy posiblemente reflejará un porcentaje inferior de aciertos. Pero es una metrica que nos vale para empezar a tomar decisiones.

# Función para evaluar una sola instancia

La función para evaluar una sola instancia utilizando un modelo preentrenado de BERT. Vamos a desglosar cada paso:

1. Preparación de datos:

*   La función recibe una instancia que contiene tres elementos clave:

> *   `context`: El contexto o el texto en el que se encuentra la pregunta.
*   `question`: La pregunta que se desea responder.
*   `given_answer`: La respuesta correcta proporcionada en la instancia (suponemos que es la primera respuesta).

2. Tokenización:

*   Primero, se tokenizan los datos utilizando el tokenizador BERT.
*   La función `tokenizer` toma la pregunta y el contexto, y devuelve una representación tokenizada de los mismos.
*   Se especifica `return_tensors='pt'` para obtener tensores de PyTorch como salida.


3. Preparación de entradas para el modelo:

*   Los datos tokenizados se convierten en tensores de PyTorch y se envían al dispositivo (CPU o GPU) especificado (variable `device`).
*   Esto es necesario para que el modelo BERT pueda procesar los datos en el mismo dispositivo.

4. Aplicación del modelo BERT:

*   Se utiliza el modelo preentrenado de BERT para procesar las entradas.
*   La línea `output = model(**inputs)` realiza la inferencia en el modelo.
*   La salida contiene dos tensores: `start_logits` y `end_logits`, que representan las puntuaciones de inicio y fin para la respuesta.

5. Obtención de la respuesta predicha:

*   Se encuentra el índice con la puntuación máxima tanto para el inicio como para el final de la respuesta.
*   Luego, se extraen los tokens correspondientes a esa ventana de índices.
*   Finalmente, se convierten los tokens en una cadena de texto (respuesta predicha).

6. Comparación con la respuesta dada:

*   La respuesta predicha se compara con la respuesta proporcionada en la instancia.
*   Si son iguales (ignorando mayúsculas y minúsculas), la función devuelve `True`, lo que indica que la predicción fue correcta.

En resumen, esta función toma una instancia con contexto, pregunta y respuesta correcta, tokeniza los datos, aplica el modelo BERT y compara la respuesta predicha con la respuesta dada. Si coinciden, devuelve `True`. Es una parte fundamental en la evaluación de modelos de procesamiento del lenguaje natural como BERT.

In [ ]:
# Función para evaluar una sola instancia
def evaluate_instance(instance, device):
    context = instance['context']
    question = instance['question']
    given_answer = instance['answers']['text'][0]  # Suponiendo que la primera respuesta sea la correcta

    # Tokenizar los datos
    inputs = tokenizer(question, context, return_tensors='pt', max_length=512, truncation=True)

    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Aplicación del modelo BERT
    with torch.no_grad():  # No es necesario calcular gradientes
        output = model(**inputs)

    # Obtén la respuesta predicha (predicted answer)
    start_idx = torch.argmax(output.start_logits)
    end_idx = torch.argmax(output.end_logits)
    predicted_answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][start_idx:end_idx + 1]))

    return predicted_answer.lower() == given_answer.lower()


# La librería `tqdm `en Python

La librería `tqdm `de Python es una librería muy útil que proporciona una barra de progreso visual y una estimación del tiempo restante para iteraciones en bucles.

Cuando utilizas tqdm en un bucle for, envuelves la iteración con la función tqdm() para mostrar una barra de progreso.

En nuestro caso, tqdm(range(total_count)) envuelve la iteración en un bucle sobre un rango del números de registros que tiene el Dataset train , mostrando una barra de progreso en la consola que se actualiza a medida que avanza el bucle.

Además de la barra de progreso, tqdm también proporciona otras características útiles, como la capacidad de mostrar el tiempo transcurrido, el tiempo restante estimado y la tasa de iteración. Esto hace que sea fácil monitorear el progreso de los bucles, especialmente cuando tienes iteraciones largas o bucles anidados.

In [ ]:
from tqdm import tqdm

# Aplicar el modelo en un conjunto de instancias

In [ ]:
# Evalúe el número de instancias
correct_count = 0
total_count = len(data['train'])
total_count

404

In [ ]:
# Iteración en un bucle con tqdm
for i in tqdm(range(total_count)):
    correct_count += evaluate_instance(data['train'][i], device)

100%|██████████| 404/404 [00:05<00:00, 73.07it/s]


In [ ]:
# Calcule y genere la precisión (accuracy)
accuracy = correct_count / total_count
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 64.85%


In [ ]:
# Evalúe el número de instancias
correct_count = 0
total_count = len(data['validation'])
total_count

102

In [ ]:
# Iteración en un bucle con tqdm
for i in tqdm(range(total_count)):
    correct_count += evaluate_instance(data['validation'][i], device)

100%|██████████| 102/102 [00:01<00:00, 57.23it/s]


In [ ]:
# Calcule y genere la precisión (accuracy)
accuracy = correct_count / total_count
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 56.86%


In [ ]:
def generate_answer(context, question, model, tokenizer, device):
    # Tokenizar los datos
    inputs = tokenizer(question, context, return_tensors='pt', max_length=512, truncation=True)

    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Aplicación del modelo BERT
    with torch.no_grad():  # No es necesario calcular gradientes
        output = model(**inputs)

    # Obtén la respuesta predicha (predicted answer)
    start_idx = torch.argmax(output.start_logits)
    end_idx = torch.argmax(output.end_logits)
    predicted_answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][start_idx:end_idx + 1]))

    return predicted_answer.lower()

# Ejemplo de uso:
context = "El Canal de Isabel II es una empresa pública española encargada de la gestión del ciclo integral del agua en la Comunidad de Madrid, su mision es principalmente el abastecimiento de la población."
question = "¿Cuál es la misión de Canal de Isabel II?"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

answer = generate_answer(context, question, model, tokenizer, device)
print("Respuesta generada por el asistente:", answer)


Respuesta generada por el asistente: su mision es principalmente el abastecimiento de la población


# Cargar el modelo que guarde despues del entrenamiento

Para cargar un modelo que ha sido guardado utilizando `trainer.save_model(),` puedes seguir estos pasos utilizando la biblioteca transformers de Hugging Face en Python:

In [ ]:
# Ruta donde se guardó el modelo
ruta_modelo = '/content/drive/MyDrive/MODELOS/OptimizadorAdamW'
ruta_modelo

'/content/drive/MyDrive/MODELOS/OptimizadorAdamW'

In [ ]:
# Cargar el modelo y el tokenizador
tokenizer2 = AutoTokenizer.from_pretrained(ruta_modelo)
tokenizer2

BertTokenizerFast(name_or_path='/content/drive/MyDrive/MODELOS/OptimizadorAdamW', vocab_size=31002, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	5: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [ ]:
# Cargar el modelo
model2 = AutoModelForQuestionAnswering.from_pretrained(ruta_modelo)


Some weights of the model checkpoint at /content/drive/MyDrive/MODELOS/OptimizadorAdamW were not used when initializing BertForQuestionAnswering: ['qa_outputs.additional_layers.0.bias', 'qa_outputs.additional_layers.0.weight', 'qa_outputs.additional_layers.3.bias', 'qa_outputs.additional_layers.3.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Cuando guardas o cargas un **modelo BERT con capas adicionales agregadas** al clasificador de salida, la estructura del modelo no mostrará explícitamente esas capas adicionales en la lista de capas del modelo. Sin embargo, las capas adicionales estarán presentes en el modelo guardado y se cargarán junto con el resto del modelo al cargarlo de nuevo.

El modelo BERT preentrenado tiene una estructura fija que incluye capas específicas para la tarea preentrenada, como clasificación de texto o preguntas y respuestas. Cuando agregas capas adicionales al clasificador de salida, estas se integran en la arquitectura del modelo como módulos secuenciales o lineales, pero no se incluyen explícitamente en la estructura del modelo BERT preentrenado.

Cuando guardas el modelo usando `model.save_pretrained()` y luego lo cargas usando `AutoModelForQuestionAnswering.from_pretrained()`, las capas adicionales que se agregaron al clasificador de salida se guardarán y cargarán junto con el modelo BERT preentrenado. Sin embargo, al imprimir la estructura del modelo, solo se verán las capas preexistentes del modelo BERT original, aunque las capas adicionales estarán presentes y funcionando correctamente en el modelo cargado.

In [ ]:
model2

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elem

# Evaluar el resultado de los datos de ejemplo con el modelo 2 guardado y cargado

Para evaluar los resultados del asistente basado en un modelo preentrenado de BERT con fine-tuning en un conjunto de datos etiquetado, podemos utilizar varias métricas.

1. Exactitud (**Accuracy**): Esta métrica mide la proporción de respuestas correctas proporcionadas por el asistente en comparación con las respuestas etiquetadas en el conjunto de datos. La exactitud es útil para evaluar el rendimiento general del modelo en todas las preguntas.

2. Precisión (**Precision**) y Recuperación (**Recall**): Estas métricas son útiles para evaluar el rendimiento del asistente en preguntas específicas. La precisión mide la proporción de respuestas correctas entre todas las respuestas proporcionadas por el asistente para una pregunta en particular, mientras que la recuperación mide la proporción de respuestas correctas proporcionadas por el asistente en relación con todas las respuestas correctas posibles para esa pregunta en el conjunto de datos.

3. **F1-Score**: Esta métrica es la media armónica de precisión y recuperación y proporciona una medida única del rendimiento del asistente en una pregunta específica. Es útil cuando se desea equilibrar la precisión y la recuperación.

In [ ]:
# Función para evaluar una sola instancia del modelo2
def evaluate_instance_model2(instance, device):
    context = instance['context']
    question = instance['question']
    given_answer = instance['answers']['text'][0]  # Suponiendo que la primera respuesta sea la correcta

    # Tokenizar2 los datos
    inputs = tokenizer2(question, context, return_tensors='pt', max_length=512, truncation=True)

    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Aplicación del modelo BERT
    with torch.no_grad():  # No es necesario calcular gradientes
        output = model2(**inputs)

    # Obtén la respuesta predicha (predicted answer)
    start_idx = torch.argmax(output.start_logits)
    end_idx = torch.argmax(output.end_logits)
    predicted_answer = tokenizer2.convert_tokens_to_string(tokenizer2.convert_ids_to_tokens(inputs['input_ids'][0][start_idx:end_idx + 1]))

    return predicted_answer.lower() == given_answer.lower()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model2.to(device)
inputs = {k: v.to(device) for k, v in inputs.items()}

# Evaluación de las respuestas sobre el conjunto de datos de train

In [ ]:
def evaluate_instance2(instance, generated_response):
    # Obtén la pregunta y la respuesta de referencia de la instancia
    question = instance['question']
    reference_response = instance['answers']['text'][0]  # Suponiendo que la primera respuesta sea la correcta

    # Calcula la precisión, el recall y el F1-Score
    precision = len(set(generated_response.split()) & set(reference_response.split())) / len(generated_response.split())
    recall = len(set(generated_response.split()) & set(reference_response.split())) / len(reference_response.split())
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    # Calcula la precisión (accuracy)
    accuracy = 1 if generated_response.lower() == reference_response.lower() else 0

    return {
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score,
        'accuracy': accuracy
    }

# Evaluación del conjunto de datos
total_count = len(data['train'])
precision_total = 0
recall_total = 0
f1_score_total = 0
accuracy_total = 0

for i in tqdm(range(total_count)):
    instance = data['train'][i]
    generated_response = generate_answer(context, question, model2, tokenizer2, device)
    evaluation_result = evaluate_instance2(instance, generated_response)

    # Agrega los resultados de precisión, recall, F1-Score y accuracy a los totales
    precision_total += evaluation_result['precision']
    recall_total += evaluation_result['recall']
    f1_score_total += evaluation_result['f1_score']
    accuracy_total += evaluation_result['accuracy']

# Calcula las métricas promedio
precision_avg = precision_total / total_count
recall_avg = recall_total / total_count
f1_score_avg = f1_score_total / total_count
accuracy_avg = accuracy_total / total_count


100%|██████████| 404/404 [00:05<00:00, 70.18it/s]


In [ ]:
# Imprime los resultados
print(f'Precision: {precision_avg}')
print(f'Recall: {recall_avg}')
print(f'F1-Score: {f1_score_avg}')
print(f'Accuracy: {accuracy}')

Precision: 0.2021452145214528
Recall: 0.0865754383946521
F1-Score: 0.1143769991589111
Accuracy: 0.5686274509803921


Este código asegura que la precisión, recall y F1-Score se calculen correctamente sin importar la longitud de las respuestas generadas y de referencia. Las respuestas se convierten a minúsculas antes de calcular las métricas para evitar discrepancias debido a diferencias de capitalización.

1. Precisión (Precision): La precisión mide la fracción de instancias recuperadas que son relevantes. En este caso, se calcula la precisión dividiendo el tamaño de la intersección entre las palabras de la respuesta generada y las palabras de la respuesta de referencia entre el tamaño de la respuesta generada.

2. Recall: El recall mide la fracción de instancias relevantes que han sido recuperadas sobre el total de instancias relevantes. De manera similar a la precisión, estás calculando el recall dividiendo el tamaño de la intersección entre las palabras de la respuesta generada y las palabras de la respuesta de referencia entre el tamaño de la respuesta de referencia.

3. F1-Score: El F1-Score es la media harmónica de precisión y recall y se utiliza para proporcionar un equilibrio entre precisión y recall. E

4. Accuracy: La precisión (accuracy) simplemente mide la fracción de instancias clasificadas correctamente sobre el total de instancias. En este caso, se compara la respuesta generada con la respuesta de referencia. Es importante destacar que el accuracy no tiene en cuenta la relación token a token entre la respuesta generada y la respuesta de referencia, lo que puede explicar por qué los resultados de precisión, recall y F1-Score no coinciden con el accuracy.

# Función para calcular las métricas de precisión, recall y F1-Score

In [ ]:
# Función para calcular las métricas de precisión, recall y F1-Score
def calculate_metrics(reference_response, generated_response):
    # Convertir las respuestas de referencia y generadas a conjuntos de palabras únicas en minúsculas
    reference_words = set(reference_response.lower().split())
    generated_words = set(generated_response.lower().split())

    # Precisión: cantidad de palabras correctas en la respuesta generada dividido por el total de palabras generadas
    precision = len(reference_words.intersection(generated_words)) / len(generated_words) if len(generated_words) > 0 else 0

    # Recall: cantidad de palabras correctas en la respuesta generada dividido por el total de palabras de referencia
    recall = len(reference_words.intersection(generated_words)) / len(reference_words) if len(reference_words) > 0 else 0

    # F1-Score: medida armónica entre precisión y recall
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1_score

# Evaluación de las respuestas se hace sobre el conjunto de datos de validación

El conjunto de datos de **validación** se utiliza para evaluar el rendimiento del modelo después de haber sido entrenado en el conjunto de datos de entrenamiento.

El conjunto de datos de validación se utiliza para medir cómo se generaliza el modelo a datos no vistos durante el entrenamiento y, por lo tanto, proporciona una estimación del rendimiento del modelo en datos del mundo real.


In [ ]:
# Función para evaluar una instancia y calcular las métricas
def evaluate_instance2(instance, generated_response):
    # Obtener la pregunta y la respuesta de referencia de la instancia
    question = instance['question']
    reference_response = instance['answers']['text'][0]  # Suponiendo que la primera respuesta sea la correcta

    # Calcular las métricas de precisión, recall y F1-Score
    precision, recall, f1_score = calculate_metrics(reference_response, generated_response)

    # Calcular la precisión (accuracy)
    accuracy = 1 if generated_response.lower() == reference_response.lower() else 0

    return {
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score,
        'accuracy': accuracy
    }

# Evaluación del conjunto de datos
total_count = len(data['validation'])
precision_total = 0
recall_total = 0
f1_score_total = 0
accuracy_total = 0

# Iterar sobre todas las instancias del conjunto de datos de validación
for i in tqdm(range(total_count)):
    instance = data['validation'][i]

    # Generar una respuesta para la instancia actual
    generated_response = generate_answer(context, question, model2, tokenizer2, device)

    # Evaluar la instancia y calcular las métricas
    evaluation_result = evaluate_instance2(instance, generated_response)

    # Agregar los resultados de precisión, recall, F1-Score y accuracy a los totales
    precision_total += evaluation_result['precision']
    recall_total += evaluation_result['recall']
    f1_score_total += evaluation_result['f1_score']
    accuracy_total += evaluation_result['accuracy']

# Calcular las métricas promedio dividiendo la suma total por el número total de instancias
precision_avg = precision_total / total_count
recall_avg = recall_total / total_count
f1_score_avg = f1_score_total / total_count
accuracy_avg = accuracy_total / total_count


100%|██████████| 102/102 [00:01<00:00, 93.31it/s]


In [ ]:
# Imprime los resultados
print(f'Precision: {precision_avg}')
print(f'Recall: {recall_avg}')
print(f'F1-Score: {f1_score_avg}')
print(f'Accuracy: {accuracy}')

Precision: 0.21023965141612175
Recall: 0.11000556883385948
F1-Score: 0.13584312213988756
Accuracy: 0.5686274509803921


# Función para calcular las métricas de precisión y recall con umbral

*   Precision: La proporción de tokens correctos en la respuesta generada dividido por el total de tokens generados.
*   Recall: La proporción de tokens correctos en la respuesta generada dividido por el total de tokens de referencia.

Los umbrales de precisión y recall se utilizan para determinar si los valores calculados superan ciertos umbrales predefinidos, y se utilizan para asignar un valor binario (1 o 0) a precision y recall.  Umbral definido arbitrariamente: `threhold = 0.15 `

In [ ]:
def calculate_metrics(reference_response, generated_response):
    # Convertir las respuestas de referencia y generadas a listas de tokens y convertir a minúsculas
    reference_tokens = reference_response.lower().split()
    generated_tokens = generated_response.lower().split()

    # Umbral definido arbitrariamente
    threhold = 0.15

    # Proporción de tokens correctos en la respuesta generada dividido por el total de tokens generados
    precision = len(set(reference_tokens).intersection(generated_tokens)) / len(generated_tokens)

    # Umbral de Precision
    threhold = 0.15  # Umbral definido arbitrariamente
    umbralPrecision = precision > threhold

    # Precision (1 si la precisión supera el umbral, de lo contrario 0)
    precision = 1 if umbralPrecision else 0

    # Proporción de tokens correctos en la respuesta generada dividido por el total de tokens de referencia
    recall = len(set(reference_tokens).intersection(generated_tokens)) / len(reference_tokens)

    # Umbral de Recall
    umbralRecall = recall > threhold

    # Recall (1 si el recall supera el umbral, de lo contrario 0)
    recall = 1 if umbralRecall else 0

    # Devuelve las métricas calculadas
    return precision, recall


# Evaluación de las respuestas se hace sobre el conjunto de datos de validación. Precision y Recall con umbral.

In [ ]:
# Función para evaluar una instancia y calcular las métricas
def evaluate_instance2(instance, generated_response):
    # Obtener la pregunta y la respuesta de referencia de la instancia
    question = instance['question']
    reference_response = instance['answers']['text'][0]  # Suponiendo que la primera respuesta sea la correcta

    # Calcular las métricas de precisión, recall
    precision, recall = calculate_metrics(reference_response, generated_response)

    # Calcular la precisión (accuracy)
    accuracy = 1 if generated_response.lower() == reference_response.lower() else 0

    return {
        'precision': precision,
        'recall': recall,
        'accuracy': accuracy
    }

# Evaluación del conjunto de datos
total_count = len(data['validation'])
precision_total = 0
recall_total = 0
accuracy_total = 0

# Iterar sobre todas las instancias del conjunto de datos de validación
for i in tqdm(range(total_count)):
    instance = data['validation'][i]

    # Generar una respuesta para la instancia actual
    generated_response = generate_answer(context, question, model2, tokenizer2, device)

    # Evaluar la instancia y calcular las métricas
    evaluation_result = evaluate_instance2(instance, generated_response)

    # Agregar los resultados de precisión, recall, F1-Score y accuracy a los totales
    precision_total += evaluation_result['precision']
    recall_total += evaluation_result['recall']
    accuracy_total += evaluation_result['accuracy']

# Calcular las métricas promedio dividiendo la suma total por el número total de instancias
precision_avg = precision_total / total_count
recall_avg = recall_total / total_count
f1_score_avg = 2 * (precision_avg * recall_avg ) / (precision_avg + recall_avg )
accuracy_avg = accuracy_total / total_count

100%|██████████| 102/102 [00:01<00:00, 90.13it/s]


In [ ]:
# Imprime los resultados
print(f'Precision: {precision_avg}')
print(f'Recall: {recall_avg}')
print(f'F1-Score: {f1_score_avg}')
print(f'Accuracy: {accuracy}')

Precision: 0.5980392156862745
Recall: 0.13725490196078433
F1-Score: 0.22326797385620914
Accuracy: 0.5686274509803921


# Tokenizar los datos de con el nuevo modelo fine-tuning

In [ ]:
instance = data['train'][10]
context = instance['context']
question = instance['question']

In [ ]:
context

'En tu factura podrás distinguir cuatro conceptos facturados: aducción, distribución, depuración y alcantarillado. Cada uno de ellos se ve afectado por unas ratios diferentes y consta de dos partes. Por un lado, una cuota de servicio que es fija y se factura independientemente de que exista o no consumo. Por otro, una parte variable que dependerá del consumo de agua realizado en el periodo que se factura.'

In [ ]:
question

'¿Cómo se compone la factura del agua?'

In [ ]:
# Tokenizar la pregunta + contexto con tokenizer2
inputs1 = tokenizer2(question, context, return_tensors='pt', max_length=512, truncation=True)

inputs1

{'input_ids': tensor([[    4,  1063,  1475,  1057, 14857,  1032, 17440,  1081,  2326,  1059,
             5,  1035,  1203, 17440,  8791, 14298,  2427, 10996, 17440,  1443,
           995,  6057,  5051,  1019,  5370,  1019, 30031,  1040, 19670, 13644,
          1008,  1748,  1614,  1009,  1792,  1057,  1480, 12878,  1076,  3063,
         30417,  1014,  3055,  1040,  8163,  1009,  1411,  2438,  1008,  1076,
          1044,  2466,  1019,  1091, 13431,  1009,  2809,  1041,  1028, 11239,
          1040,  1057, 17440,  4794,  1200,  1009,  1041, 14975,  1068,  1054,
          5826,  1008,  1076,  1811,  1019,  1091,  1514, 14290,  1041, 22036,
          1081,  5826,  1009,  2326,  5753,  1035,  1039,  6083,  1041,  1057,
         17440,  1008,     5]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [ ]:
inputs = inputs1

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model2.to(device)
inputs = {k: v.to(device) for k, v in inputs.items()}

In [ ]:
# Obtener la salida del modelo
with torch.no_grad():
    output = model2(**inputs)

In [ ]:
# Obtener la respuesta prevista
start_idx = torch.argmax(output.start_logits)
end_idx = torch.argmax(output.end_logits)

predicted_answer = tokenizer2.convert_tokens_to_string(tokenizer2.convert_ids_to_tokens(inputs['input_ids'][0][start_idx:end_idx + 1]))

In [ ]:
predicted_answer, start_idx, end_idx, start_idx.item(), end_idx.item()

('aducción, distribución, depuración y alcantarillado',
 tensor(21, device='cuda:0'),
 tensor(29, device='cuda:0'),
 21,
 29)

In [ ]:
instance['answers']

{'answer_start': [61],
 'text': ['Aducción, distribución, depuración y alcantarillado']}

In [ ]:
given_answer = instance['answers']['text'][0]  # Suponiendo que la primera respuesta sea la correcta
given_answer_start = instance['answers']['answer_start'][0]
given_answer, given_answer_start

('Aducción, distribución, depuración y alcantarillado', 61)

In [ ]:
correct = (predicted_answer.lower() == given_answer.lower())
evaluation = 'Correcto' if correct else f'Incorrecto (Predicción: {predicted_answer}, Solución: {given_answer})'

print(evaluation)

Correcto


# Realizar una inferencia con el modelo entrenado con fine-tuning

Para realizar la **inferencia** utilizando el modelo cargado **model2**, primero necesitas tokenizar los textos de contexto y pregunta utilizando el tokenizer, luego alimentar los tokens al modelo para obtener las respuestas.

In [ ]:
# Texto de contexto y pregunta
contexto = "Si ya estás dado de alta en la Oficina Virtual, además de todas las gestiones recogidas en el apartado anterior, puedes acceder a las siguientes: 1.Trámites que afectan a tus contratos: Actualización de los datos de contacto y modificación de los datos de pago. 2.Asociados a tus facturas: Consultar facturas anteriores, reclamar alguna factura, enviarnos un justificante de pago, solicitar una devolución de saldo y descargar un listado de tus facturas. 3.Relacionadas con cancelaciones: Dar de baja un suministro y darte de baja como usuario de la Oficina virtual. 4.Otras gestiones: Consultar el estado de tus quejas y acceder al arbitraje de consumo."
pregunta = "¿Qué trámites puedo realizar en relación con mis facturas si estoy registrado en la Oficina Virtual?"

# Tokenizar el contexto y la pregunta
tokens = tokenizer2.encode_plus(pregunta, contexto, return_tensors="pt", max_length=512, truncation=True)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model2.to(device)
inputs = {k: v.to(device) for k, v in inputs.items()}

In [ ]:
# Realizar la inferencia
device = torch.device("cpu")

# Realizar la inferencia
with torch.no_grad():
    outputs = model2.to(device)(**tokens.to(device))

# Obtener los logits de inicio y fin de la respuesta
start_logits = outputs.start_logits
end_logits = outputs.end_logits

# Encontrar el índice del token con el valor más alto en los logits de inicio y fin
inicio_respuesta = torch.argmax(start_logits)
fin_respuesta = torch.argmax(end_logits)

# Decodificar la respuesta
respuesta = tokenizer2.decode(tokens['input_ids'][0][inicio_respuesta:fin_respuesta+1])

print("Respuesta:", respuesta)

Respuesta: consultar facturas anteriores, reclamar alguna factura, enviarnos un justificante de pago, solicitar una devolución de saldo y descargar un listado de tus facturas


In [ ]:
# Texto de contexto y pregunta
contexto = "En tu factura podrás distinguir cuatro conceptos facturados: aducción, distribución, depuración y alcantarillado. Cada uno de ellos se ve afectado por unas ratios diferentes y consta de dos partes. Por un lado, una cuota de servicio que es fija y se factura independientemente de que exista o no consumo. Por otro, una parte variable que dependerá del consumo de agua realizado en el periodo que se factura. Bimestralmente. Es decir, cada dos meses. Una vez emitida la factura, tienes un periodo voluntario de pago de 30 días naturales. El Reglamento para el Servicio y Distribución de las Aguas de Canal de Isabel II (Decreto 2922/1975, de 31 de octubre) establece varios métodos para facturar el consumo de agua: 1.Estimación de consumos (EST): este método se utiliza cuando no es posible acceder al contador en la fecha en que debe realizarse la lectura. Su cálculo se efectúa de acuerdo con el consumo medio diario de los dos periodos análogos precedentes y, en caso de disponerse solo del histórico de consumos de un año, la estimación se realizará de acuerdo con el consumo medio diario del periodo análogo precedente. El consumo facturado de esta forma se considera a cuenta. Por tanto, una vez que se tome la lectura real del contador, los metros cúbicos facturados de esta forma se descontarán del consumo realizado. 2.Evaluación de consumos (EV): para facturar los metros cúbicos mediante este método, se debe dar la circunstancia de que, aun teniendo acceso al contador, no se pueda recoger la lectura porque exista una anomalía que requiera su sustitución. El cálculo del consumo se realiza de forma análoga a la estimación. 3.Diferencia de lecturas del aparato de medida (DFI): diferencia de la lectura tomada en el periodo anterior y la actual. Puedes consultar las tarifas aplicables para la facturación del servicio de suministro de agua en la sección de tarifas de esta web. Estudiamos individualmente las solicitudes, y tras su análisis, concedemos, de acuerdo con una serie de criterios, el número de plazos e interés correspondiente según el importe y las características del cliente. Según establece el Reglamento para el Servicio y Distribución de las Aguas de Canal de Isabel II, (Decreto 2922/1975, de 31 de octubre) podríamos suspender el suministro de agua si transcurridos 30 días naturales desde la emisión de la factura, esta no ha sido abonada. El corte del suministro implicaría continuar facturando las cuotas de servicio. El restablecimiento del servicio se realizará una vez liquidada la deuda, así como el importe del restablecimiento. Transcurridos tres meses sin que se haya producido el pago, Canal puede resolver el contrato y proceder a la condena de la acometida (retirada de la instalación). Para volver a tener agua habrá que realizar una nueva contratación."
pregunta = "¿Qué conceptos hay en la factura?"

# Tokenizar el contexto y la pregunta
tokens = tokenizer2.encode_plus(pregunta, contexto, return_tensors="pt", max_length=512, truncation=True)

In [ ]:
# Realizar la inferencia
device = torch.device("cpu")

# Realizar la inferencia
with torch.no_grad():
    outputs = model2.to(device)(**tokens.to(device))

# Realizar la inferencia
#with torch.no_grad():
#    outputs = model2(**tokens)

# Obtener los logits de inicio y fin de la respuesta
start_logits = outputs.start_logits
end_logits = outputs.end_logits

# Encontrar el índice del token con el valor más alto en los logits de inicio y fin
inicio_respuesta = torch.argmax(start_logits)
fin_respuesta = torch.argmax(end_logits)

# Decodificar la respuesta
respuesta = tokenizer2.decode(tokens['input_ids'][0][inicio_respuesta:fin_respuesta+1])

print("Respuesta:", respuesta)

Respuesta: aducción, distribución, depuración y alcantarillado


In [ ]:
# Texto de contexto y pregunta
contexto = "En tu factura podrás distinguir cuatro conceptos facturados: aducción, distribución, depuración y alcantarillado. Cada uno de ellos se ve afectado por unas ratios diferentes y consta de dos partes. Por un lado, una cuota de servicio que es fija y se factura independientemente de que exista o no consumo. Por otro, una parte variable que dependerá del consumo de agua realizado en el periodo que se factura. Bimestralmente. Es decir, cada dos meses. Una vez emitida la factura, tienes un periodo voluntario de pago de 30 días naturales. El Reglamento para el Servicio y Distribución de las Aguas de Canal de Isabel II (Decreto 2922/1975, de 31 de octubre) establece varios métodos para facturar el consumo de agua: 1.Estimación de consumos (EST): este método se utiliza cuando no es posible acceder al contador en la fecha en que debe realizarse la lectura. Su cálculo se efectúa de acuerdo con el consumo medio diario de los dos periodos análogos precedentes y, en caso de disponerse solo del histórico de consumos de un año, la estimación se realizará de acuerdo con el consumo medio diario del periodo análogo precedente. El consumo facturado de esta forma se considera a cuenta. Por tanto, una vez que se tome la lectura real del contador, los metros cúbicos facturados de esta forma se descontarán del consumo realizado. 2.Evaluación de consumos (EV): para facturar los metros cúbicos mediante este método, se debe dar la circunstancia de que, aun teniendo acceso al contador, no se pueda recoger la lectura porque exista una anomalía que requiera su sustitución. El cálculo del consumo se realiza de forma análoga a la estimación. 3.Diferencia de lecturas del aparato de medida (DFI): diferencia de la lectura tomada en el periodo anterior y la actual. Puedes consultar las tarifas aplicables para la facturación del servicio de suministro de agua en la sección de tarifas de esta web. Estudiamos individualmente las solicitudes, y tras su análisis, concedemos, de acuerdo con una serie de criterios, el número de plazos e interés correspondiente según el importe y las características del cliente. Según establece el Reglamento para el Servicio y Distribución de las Aguas de Canal de Isabel II, (Decreto 2922/1975, de 31 de octubre) podríamos suspender el suministro de agua si transcurridos 30 días naturales desde la emisión de la factura, esta no ha sido abonada. El corte del suministro implicaría continuar facturando las cuotas de servicio. El restablecimiento del servicio se realizará una vez liquidada la deuda, así como el importe del restablecimiento. Transcurridos tres meses sin que se haya producido el pago, Canal puede resolver el contrato y proceder a la condena de la acometida (retirada de la instalación). Para volver a tener agua habrá que realizar una nueva contratación."
pregunta = "¿Qué cuatro conceptos hay en la factura?"

# Tokenizar el contexto y la pregunta
tokens = tokenizer2.encode_plus(pregunta, contexto, return_tensors="pt", max_length=512, truncation=True)

In [ ]:
# Realizar la inferencia
device = torch.device("cpu")

# Realizar la inferencia
with torch.no_grad():
    outputs = model2.to(device)(**tokens.to(device))

# Obtener los logits de inicio y fin de la respuesta
start_logits = outputs.start_logits
end_logits = outputs.end_logits

# Encontrar el índice del token con el valor más alto en los logits de inicio y fin
inicio_respuesta = torch.argmax(start_logits)
fin_respuesta = torch.argmax(end_logits)

# Decodificar la respuesta
respuesta = tokenizer2.decode(tokens['input_ids'][0][inicio_respuesta:fin_respuesta+1])

print("Respuesta:", respuesta)

Respuesta: aducción, distribución, depuración y alcantarillado


In [ ]:
pregunta = "¿Cuanto es el tiempo voluntario de pago?"

# Tokenizar el contexto y la pregunta
tokens = tokenizer2.encode_plus(pregunta, contexto, return_tensors="pt", max_length=512, truncation=True)

In [ ]:
# Realizar la inferencia
device = torch.device("cpu")

# Realizar la inferencia
with torch.no_grad():
    outputs = model2.to(device)(**tokens.to(device))

# Obtener los logits de inicio y fin de la respuesta
start_logits = outputs.start_logits
end_logits = outputs.end_logits

# Encontrar el índice del token con el valor más alto en los logits de inicio y fin
inicio_respuesta = torch.argmax(start_logits)
fin_respuesta = torch.argmax(end_logits)

# Decodificar la respuesta
respuesta = tokenizer2.decode(tokens['input_ids'][0][inicio_respuesta:fin_respuesta+1])

print("Respuesta:", respuesta)

Respuesta: 30 días naturales


In [ ]:
# Texto de contexto y pregunta
contexto = "En primer lugar, averigua si la falta de agua afecta a todos los puntos de agua de tu casa. Si no, pregunta a algún vecino si tiene agua. En caso de que sí tenga, comprueba que la llave de paso interior de tu vivienda esté completamente abierta y verifica que el contador o el equipo de medición tenga las dos llaves de paso abiertas (es decir, giradas completamente a la izquierda). En la batería de contadores hay un plano donde podrás encontrar la ubicación de tu contador. Si después de haber hecho estas comprobaciones sigues sin agua, ponte en contacto con nosotros."
pregunta = "¿Qué debo revisar si tengo falta de agua en mi vivienda?"

# Tokenizar el contexto y la pregunta
tokens = tokenizer2.encode_plus(pregunta, contexto, return_tensors="pt", max_length=512, truncation=True)

In [ ]:
# Realizar la inferencia
device = torch.device("cpu")

# Realizar la inferencia
with torch.no_grad():
    outputs = model2.to(device)(**tokens.to(device))

# Obtener los logits de inicio y fin de la respuesta
start_logits = outputs.start_logits
end_logits = outputs.end_logits

# Encontrar el índice del token con el valor más alto en los logits de inicio y fin
inicio_respuesta = torch.argmax(start_logits)
fin_respuesta = torch.argmax(end_logits)

# Decodificar la respuesta
respuesta = tokenizer2.decode(tokens['input_ids'][0][inicio_respuesta:fin_respuesta+1])

print("Respuesta:", respuesta)

Respuesta: comprueba que la llave de paso interior de tu vivienda esté completamente abierta y verifica que el contador o el equipo de medición tenga las dos llaves de paso abiertas ( es decir, giradas completamente a la izquierda )


In [ ]:
# Texto de contexto y pregunta
contexto = "En primer lugar, averigua si la falta de agua afecta a todos los puntos de agua de tu casa. Si no, pregunta a algún vecino si tiene agua. En caso de que sí tenga, comprueba que la llave de paso interior de tu vivienda esté completamente abierta y verifica que el contador o el equipo de medición tenga las dos llaves de paso abiertas (es decir, giradas completamente a la izquierda). En la batería de contadores hay un plano donde podrás encontrar la ubicación de tu contador. Si después de haber hecho estas comprobaciones sigues sin agua, ponte en contacto con nosotros."
pregunta = "¿Qué debo hacer si no tengo agua en mi casa?"

# Tokenizar el contexto y la pregunta
tokens = tokenizer2.encode_plus(pregunta, contexto, return_tensors="pt", max_length=512, truncation=True)

In [ ]:
# Realizar la inferencia
device = torch.device("cpu")

# Realizar la inferencia
with torch.no_grad():
    outputs = model2.to(device)(**tokens.to(device))

# Obtener los logits de inicio y fin de la respuesta
start_logits = outputs.start_logits
end_logits = outputs.end_logits

# Encontrar el índice del token con el valor más alto en los logits de inicio y fin
inicio_respuesta = torch.argmax(start_logits)
fin_respuesta = torch.argmax(end_logits)

# Decodificar la respuesta
respuesta = tokenizer2.decode(tokens['input_ids'][0][inicio_respuesta:fin_respuesta+1])

print("Respuesta:", respuesta)

Respuesta: en primer lugar, averigua si la falta de agua afecta a todos los puntos de agua de tu casa. si no, pregunta a algún vecino si tiene agua


In [ ]:
pregunta = "¿Qué debo hacer si sigo sin agua después de verificar el contador?"

# Tokenizar el contexto y la pregunta
tokens = tokenizer2.encode_plus(pregunta, contexto, return_tensors="pt", max_length=512, truncation=True)


In [ ]:
# Realizar la inferencia
device = torch.device("cpu")

# Realizar la inferencia
with torch.no_grad():
    outputs = model2.to(device)(**tokens.to(device))

# Obtener los logits de inicio y fin de la respuesta
start_logits = outputs.start_logits
end_logits = outputs.end_logits

# Encontrar el índice del token con el valor más alto en los logits de inicio y fin
inicio_respuesta = torch.argmax(start_logits)
fin_respuesta = torch.argmax(end_logits)

# Decodificar la respuesta
respuesta = tokenizer2.decode(tokens['input_ids'][0][inicio_respuesta:fin_respuesta+1])

print("Respuesta:", respuesta)

Respuesta: ponte en contacto con nosotros


In [ ]:
# Texto de contexto y pregunta
contexto = "Puedes elegir siete vías diferentes para contactar con nosotros: La manera más rápida es utilizar el chat de esta Oficina Virtual. Si prefieres hablar por teléfono, llama a nuestro teléfono gratuito 900 365 365 en horario de 08:00 a 20:00. Si necesitas comunicar una incidencia o avería, el servicio se encuentra operativo las 24 horas del día, los 365 días del año. Otra opción telefónica es que te llamemos nosotros. Solo tienes que indicar tu número de teléfono y el rango horario que prefieras. Puedes utilizar un formulario de contacto. Tienes la posibilidad de escribirnos un correo electrónico a la dirección: clientes@canaldeisabelsegunda.es. Puedes pedir cita para que te atendamos en nuestros centros de atención al cliente. Y si prefieres métodos tradicionales, puedes enviar una comunicación escrita por correo ordinario o fax: Att. Registro General. C/ Santa Engracia, 125 (28003 - Madrid). Horario: de lunes a viernes de 8:30 a 14:00 horas. Fax: 915 451 430. Independientemente del canal de comunicación que prefieras utilizar, no olvides indicarnos como referencia tu número de contrato o la dirección de la finca objeto de la solicitud, así como un teléfono de contacto."
pregunta = "¿Cuál es el teléfono de atención al cliente?"


# Tokenizar el contexto y la pregunta
tokens = tokenizer2.encode_plus(pregunta, contexto, return_tensors="pt", max_length=512, truncation=True)

In [ ]:
# Realizar la inferencia
device = torch.device("cpu")

# Realizar la inferencia
with torch.no_grad():
    outputs = model2.to(device)(**tokens.to(device))

# Obtener los logits de inicio y fin de la respuesta
start_logits = outputs.start_logits
end_logits = outputs.end_logits

# Encontrar el índice del token con el valor más alto en los logits de inicio y fin
inicio_respuesta = torch.argmax(start_logits)
fin_respuesta = torch.argmax(end_logits)

# Decodificar la respuesta
respuesta = tokenizer2.decode(tokens['input_ids'][0][inicio_respuesta:fin_respuesta+1])

print("Respuesta:", respuesta)

Respuesta: teléfono gratuito 900 365 365


# Hacer la inferencia sin contexto, recorriendo el dataset y buscar la pregunta (question) que mas se asemeja

In [ ]:
from scipy.spatial.distance import cosine

In [ ]:
def predict_answer_select_context(question):

  # Tokenizar la pregunta
  inputs = tokenizer2(question, return_tensors='pt', max_length=512, truncation=True)

  # Buscar la pregunta que más se asemeja
  tokenized_question = inputs['input_ids'][0]
  menor_distancia = 1
  contexto_mas_similar = None

  for example in list(data['train']) + (list(data['validation'])):
    train_question = example['question']
    inputs_train = tokenizer2(train_question, return_tensors='pt', max_length=512, truncation=True)
    tokenized_train_question = inputs_train['input_ids'][0]

    padding_length = abs(len(tokenized_train_question) - len(tokenized_question))
    if len(tokenized_train_question) > len(tokenized_question):
      padded_question = np.pad(tokenized_question, (0, padding_length))
      padded_train_question = tokenized_train_question
    else:
      padded_question = tokenized_question
      padded_train_question = np.pad(tokenized_train_question, (0, padding_length))

    # calcula la distancia del coseno
    cosine_distance = abs(cosine(padded_question, padded_train_question))

    # mientras más cercano a 0 sea la distancia entre los dos vector mayor será la similitud
    if cosine_distance < menor_distancia:
      menor_distancia = cosine_distance
      contexto_mas_similar = example

  return contexto_mas_similar

In [ ]:
question = '¿Donde está el contador?'

In [ ]:
contexto_mas_similar = predict_answer_select_context(question)
contexto_mas_similar

{'id': '114',
 'title': 'Preguntas frecuentes: Averías',
 'context': 'En primer lugar, averigua si la falta de agua afecta a todos los puntos de agua de tu casa. Si no, pregunta a algún vecino si tiene agua. En caso de que sí tenga, comprueba que la llave de paso interior de tu vivienda esté completamente abierta y verifica que el contador o el equipo de medición tenga las dos llaves de paso abiertas (es decir, giradas completamente a la izquierda). En la batería de contadores hay un plano donde podrás encontrar la ubicación de tu contador. Si después de haber hecho estas comprobaciones sigues sin agua, ponte en contacto con nosotros.',
 'question': 'Si quiero encontrar el contador, ¿qué debo hacer?',
 'answers': {'answer_start': [384],
  'text': ['En la batería de contadores hay un plano donde podrás encontrar la ubicación de tu contador']}}

In [ ]:
contexto_mas_similar['question']

'Si quiero encontrar el contador, ¿qué debo hacer?'

In [ ]:
contexto_mas_similar['answers']

{'answer_start': [384],
 'text': ['En la batería de contadores hay un plano donde podrás encontrar la ubicación de tu contador']}

In [ ]:
contexto_mas_similar['answers']['answer_start'][0]

384

In [ ]:
contexto_mas_similar['answers']['text'][0]

'En la batería de contadores hay un plano donde podrás encontrar la ubicación de tu contador'

In [ ]:
question = "¿Qué puedo avisar al servicio de incidencias?"

In [ ]:
contexto_mas_similar = predict_answer_select_context(question)
contexto_mas_similar['answers']['text'][0]

'Puedes enviar una comunicación escrita por correo ordinario o fax: Att. Registro General. C/ Santa Engracia, 125 (28003 - Madrid). Horario: de lunes a viernes de 8:30 a 14:00 horas. Fax: 915 451 430'

In [ ]:
question = "¿Cada cuánto tiempo recibo mi factura?"

In [ ]:
contexto_mas_similar = predict_answer_select_context(question)
contexto_mas_similar['answers']['text'][0]

'Bimestralmente'

In [ ]:
question = "¿Qué conceptos hay en la factura?"

In [ ]:
contexto_mas_similar = predict_answer_select_context(question)
contexto_mas_similar['answers']['text'][0]

'Aducción, distribución, depuración y alcantarillado'

# Contexto completo para inferencia

Este código recorre cada ejemplo del conjunto de datos, extrae la pregunta y el contexto de cada ejemplo y las concatena en la variable contexto, separadas por un espacio. Al finalizar este bucle, la variable contexto contendrá el contexto deseado, donde cada pregunta está seguida de su contexto. Se puede utilizar esta variable como contexto en la inferencia con el modelo fine-tuneado.

In [ ]:
# Inicializar la variable de contexto
contexto = ""

# Construir el contexto con cada pregunta seguida de su respuesta
for example in dataset:
    pregunta = example['question']
    respuesta = example['context']
    contexto += f"{pregunta} {respuesta} "

El contexto lo reduccimos para Averías y Contacto, para acelerar el ejecucción:

In [ ]:
# Inicializar la variable de contexto
contexto = ""

# Construir el contexto con cada pregunta seguida de su respuesta para los primeros 173 ejemplos
for idx, example in enumerate(dataset):
    if idx >= 173:
        break  # Detener el bucle después de los primeros 173 ejemplos
    pregunta = example['question']
    respuesta = example['context']
    contexto += f"{pregunta} {respuesta} "

In [ ]:
contexto

'¿Qué debo hacer si no tengo agua en mi casa? En primer lugar, averigua si la falta de agua afecta a todos los puntos de agua de tu casa. Si no, pregunta a algún vecino si tiene agua. En caso de que sí tenga, comprueba que la llave de paso interior de tu vivienda esté completamente abierta y verifica que el contador o el equipo de medición tenga las dos llaves de paso abiertas (es decir, giradas completamente a la izquierda). En la batería de contadores hay un plano donde podrás encontrar la ubicación de tu contador. Si después de haber hecho estas comprobaciones sigues sin agua, ponte en contacto con nosotros. ¿Cuáles son las acciones recomendadas si carezco de agua en mi hogar? En primer lugar, averigua si la falta de agua afecta a todos los puntos de agua de tu casa. Si no, pregunta a algún vecino si tiene agua. En caso de que sí tenga, comprueba que la llave de paso interior de tu vivienda esté completamente abierta y verifica que el contador o el equipo de medición tenga las dos l

# pipeline("question-answering")

Usamos la biblioteca Transformers de Hugging Face para responder preguntas con contexto: Para abordar problemas de generación de texto y responder preguntas con contexto, vamos considerar el uso de bibliotecas de NLP como Transformers de Hugging Face, que proporciona una amplia gama de modelos preentrenados para tareas de generación de texto y preguntas y respuestas.


Este código utiliza el modelo de pregunta-respuesta proporcionado por la biblioteca Transformers de Hugging Face y le proporciona una pregunta junto con el contexto que has definido. Luego, devuelve la respuesta obtenida del modelo.

Puedes experimentar con diferentes modelos y ajustar el contexto según sea necesario para obtener mejores resultados. Además, Hugging Face's Transformers ofrece una amplia documentación y ejemplos que pueden ayudarte a entender mejor cómo utilizar sus modelos para diferentes tareas de procesamiento de lenguaje natural.

In [ ]:
from transformers import  pipeline

En la condición if **result["score"] > 0.79**, se verifica si el score de confianza de la respuesta supera 0.79 antes de devolver la respuesta. Esto puede ser útil para filtrar respuestas potencialmente incorrectas. Sin embargo, hay que tener en cuenta que este valor puede necesitar ser ajustado según las necesidades y la precisión del modelo.

Los modelos de lenguaje natural son capaces de procesar texto en cualquier formato de capitalización. Sin embargo, hay algunos casos en los que podría influir:

1. Coincidencia exacta: Si el texto de la pregunta y el texto del contexto son idénticos en términos de capitalización, es más probable que se encuentre una coincidencia exacta y la respuesta se encuentre más fácilmente.

2. Preprocesamiento de texto: Algunos modelos o pipelines de procesamiento de lenguaje natural pueden realizar preprocesamientos específicos en el texto antes de procesarlo. Si el pipeline realiza algún tipo de normalización de texto que convierte todo el texto en minúsculas, entonces podría haber una diferencia en el procesamiento de las preguntas en mayúsculas y minúsculas.

Para asegurar de que las preguntas en mayúsculas se procesen correctamente, se pueden convertir todas las preguntas a minúsculas antes de pasarlas al modelo.

    # def answer_question(question):
       # Convertir la pregunta a minúsculas
         question = question.lower()
       # Obtener la respuesta del modelo
        result = nlp(question=question,context=context)


De esta manera, independientemente de si el usuario escribe la pregunta en mayúsculas o minúsculas, será procesada de la misma manera. Esto puede ayudar a mejorar la consistencia en la búsqueda de respuestas.

In [ ]:
# Cargar el modelo de pregunta-respuesta
nlp = pipeline("question-answering", model=model2, tokenizer=tokenizer2)

In [ ]:
def answer_question(question):
    """
    Función para responder preguntas dadas una pregunta y un contexto predefinido.
    """
    # Obtener la respuesta del modelo
    result = nlp(question=question, context=contexto)

    if result["score"] > 0.79:
       # Verificar si se encontró una respuesta
       if result['answer']:  # Si la respuesta no está vacía
           # Retornar la respuesta encontrada
           return result['answer']
       else:
           # Devolver un mensaje indicando que la pregunta debe ser reformulada
           return "Lo siento, no pude encontrar una respuesta para tu pregunta. Por favor, reformula tu pregunta sobre Averías o Contacto"

    else:
       # Devolver un mensaje indicando que la pregunta debe ser reformulada
       return "No supero el 0.79 score. Lo siento, no pude encontrar una respuesta para tu pregunta. Por favor, reformula tu pregunta sobre Averías o Contacto"


In [ ]:
pregunta = "¿Qué cuatro conceptos hay en la factura?"

In [ ]:
resp = nlp(context=contexto, question=pregunta)
resp

{'score': 0.5704760551452637,
 'start': 84774,
 'end': 84836,
 'answer': 'indicar tu número de teléfono y el rango horario que prefieras'}

In [ ]:
resp["score"]

0.5704760551452637

**Visualización del resultado en formato HTML**

In [ ]:
from IPython.core.display import HTML

In [ ]:
if resp["score"] > 0.79:
  answer = resp["answer"]
  display(HTML(f'<h2>{pregunta.upper()}</h2>'))
  marked_text = str(contexto.replace(answer, f"<mark>{answer}</mark>"))
  display(HTML(f"""<blockquote> {marked_text} </blockquote>"""))
else:
  answer = "No supero el 0.79 score. Lo siento, no pude encontrar una respuesta para tu pregunta. Por favor, reformula tu pregunta sobre Averías o Contacto"
  display(HTML(f'<h2>{pregunta.upper()}</h2>'))
  display(HTML(f'<h2>{answer}</h2>'))

In [ ]:
pregunta = " "

In [ ]:
resp = nlp(context=contexto, question=pregunta)
resp

{'score': 0.02324775978922844,
 'start': 90090,
 'end': 90119,
 'answer': 'teléfono gratuito 900 365 365'}

**Visualización del resultado en formato HTML**

In [ ]:
if resp["score"] > 0.79:
  answer = resp["answer"]
  display(HTML(f'<h2>{pregunta.upper()}</h2>'))
  marked_text = str(contexto.replace(answer, f"<mark>{answer}</mark>"))
  display(HTML(f"""<blockquote> {marked_text} </blockquote>"""))
else:
  answer = "No supero el 0.79 score. Lo siento, no pude encontrar una respuesta para tu pregunta. Por favor, reformula tu pregunta sobre Averías o Contacto"
  display(HTML(f'<h2>{pregunta.upper()}</h2>'))
  display(HTML(f'<h2>{answer}</h2>'))

In [ ]:
pregunta = "¿A qué número puedo llamar para reportar una incidencia en el suministro de agua?"
resp = nlp(context=contexto, question=pregunta)
resp

{'score': 0.9807830452919006,
 'start': 54090,
 'end': 54101,
 'answer': '900 365 365'}

**Visualización del resultado en formato HTML**

In [ ]:
if resp["score"] > 0.79:
  answer = resp["answer"]
  display(HTML(f'<h2>{pregunta.upper()}</h2>'))
  marked_text = str(contexto.replace(answer, f"<mark>{answer}</mark>"))
  display(HTML(f"""<blockquote> {marked_text} </blockquote>"""))
else:
  answer = "No supero el 0.79 score. Lo siento, no pude encontrar una respuesta para tu pregunta. Por favor, reformula tu pregunta sobre Averías o Contacto"
  display(HTML(f'<h2>{pregunta.upper()}</h2>'))
  display(HTML(f'<h2>{answer}</h2>'))

## Inferencia con pipeline("question-answering"), para preguntas que entran en nuestro contexto

In [ ]:
pregunta = "¿Cuál es la dirección de correo electrónico de Canal de Isabel II?"
resp = nlp(context=contexto, question=pregunta)
resp

{'score': 0.9735689163208008,
 'start': 85979,
 'end': 86025,
 'answer': 'la dirección: clientes@canaldeisabelsegunda.es'}

In [ ]:
pregunta = "¿Cuál es el contacto del Canal de Isabel II?"
resp = nlp(context=contexto, question=pregunta)
resp

{'score': 0.9396594762802124,
 'start': 74653,
 'end': 74699,
 'answer': 'la dirección: clientes@canaldeisabelsegunda.es'}

In [ ]:
pregunta =  "¿Cuál es el teléfono del servicio de notificaciones de incidencias en el suministro de agua?"
resp = nlp(context=contexto, question=pregunta)
resp

{'score': 0.8074807524681091,
 'start': 34744,
 'end': 34755,
 'answer': '900 365 365'}

In [ ]:
pregunta = "¿Qué puedo avisar al servicio de incidencias?"
resp = nlp(context=contexto, question=pregunta)
resp

{'score': 0.9267107248306274,
 'start': 34848,
 'end': 34870,
 'answer': 'cortes de agua o fugas'}

In [ ]:
pregunta = "¿Cuál es el teléfono de atención al cliente?"
resp = nlp(context=contexto, question=pregunta)
resp

{'score': 0.7625647783279419,
 'start': 74249,
 'end': 74260,
 'answer': '900 365 365'}

## Inferencia con pipeline("question-answering"), para preguntas que "No" entran en nuestro contexto

Ya que vamos a limitar al asistente a preguntas relacionadas con Averías o Contacto. Comprobamos el score de alguna pregunta relacionada con el Canal de Isabel II, pero con otra temática. Así podemos evaluar un score que nos asegure buenas respuestas.

In [ ]:
pregunta = "¿En qué horario puedo contactarlos?"
resp = nlp(context=contexto, question=pregunta)
resp

{'score': 0.8467395305633545,
 'start': 128831,
 'end': 128909,
 'answer': 'Solo tienes que indicar tu número de teléfono y el rango horario que prefieras'}

In [ ]:
pregunta = "¿Cada cuánto tiempo recibo mi factura?"
resp = nlp(context=contexto, question=pregunta)
resp

{'score': 0.8981596827507019,
 'start': 85130,
 'end': 85177,
 'answer': 'comunicación escrita por correo ordinario o fax'}

In [ ]:
pregunta = "¿Qué trámites puedo realizar en relación con mis facturas si estoy registrado en la Oficina Virtual?"
resp = nlp(context=contexto, question=pregunta)
resp

{'score': 0.9139502644538879,
 'start': 75844,
 'end': 75909,
 'answer': 'puedes enviar una comunicación escrita por correo ordinario o fax'}

In [ ]:
pregunta = "¿Qué cuatro conceptos hay en la factura?"
resp = nlp(context=contexto, question=pregunta)
resp

{'score': 0.5704760551452637,
 'start': 84774,
 'end': 84836,
 'answer': 'indicar tu número de teléfono y el rango horario que prefieras'}

# Transformers de Hugging Face para realizar inferencias de preguntas y respuestas

Se utiliza la biblioteca Transformers de Hugging Face para realizar inferencias de preguntas y respuestas utilizando el modelo preentrenado con fine-tuning que ha sido entrenado con datos etiquetados de preguntas y respuestas del Canal de Isabel II.

1. Carga del Modelo y Tokenizador: Utiliza la función AutoModelForQuestionAnswering.from_pretrained() para cargar un modelo preentrenado para la tarea de pregunta-respuesta, y AutoTokenizer.from_pretrained() para cargar el tokenizador asociado con el modelo. Ambos son cargados a partir del nombre del modelo especificado en la variable model_name.

2. Configuración del Pipeline: Se configura un pipeline de pregunta-respuesta utilizando la función pipeline('question-answering', model=model, tokenizer=tokenizer). Este pipeline utiliza el modelo y el tokenizador cargados previamente.

3. Definición del Contexto: Se define un contexto largo que contiene información sobre un tema específico. Este contexto se utiliza como fuente para buscar respuestas a las preguntas.

4. Función de Respuesta a Preguntas: La función answer_question(question) toma una pregunta como entrada, y utiliza el pipeline previamente configurado para encontrar la respuesta en el contexto proporcionado. La respuesta encontrada se devuelve como resultado de la función.

5. Ejecución de la Función con una Pregunta Específica: Se proporcionan  preguntas específicas ("¿Cada cuánto tiempo recibo mi factura?", "¿Cuanto es el tiempo voluntario de pago?") a la función answer_question(). Estas preguntas se pasan al modelo, junto con el contexto definido anteriormente, y el modelo devuelve la respuesta encontrada en el contexto.

En resumen, este código utiliza un modelo preentrenado para responder preguntas dadas un contexto específico, sobre  los servicios de agua en la empresa Canal de Isabel II de suministro de agua.







## Modelo preentrenado completo para responder preguntas dadas un contexto de Avería y Contacto, sobre los servicios de agua en la empresa Canal de Isabel II de suministro de agua

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

# Cargar el modelo y el tokenizador
model_name ='Antonio49/ModeloCanal'
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Configurar el pipeline para la pregunta-respuesta
nlp = pipeline('question-answering', model=model, tokenizer=tokenizer)

# Contexto integrado en el código
context = contexto

Some weights of the model checkpoint at Antonio49/ModeloCanal were not used when initializing BertForQuestionAnswering: ['qa_outputs.additional_layers.0.bias', 'qa_outputs.additional_layers.0.weight', 'qa_outputs.additional_layers.3.bias', 'qa_outputs.additional_layers.3.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def answer_question(question):
    """
    Función para responder preguntas dadas una pregunta y un contexto predefinido.
    """
    # Obtener la respuesta del modelo
    result = nlp(question=question, context=context)

    # Verificar si se encontró una respuesta
    if result['answer']:  # Si la respuesta no está vacía
        # Retornar la respuesta encontrada
        return result['answer']
    else:
        # Devolver un mensaje indicando que la pregunta debe ser reformulada
        return "Lo siento, no pude encontrar una respuesta para tu pregunta. Por favor, reformula tu pregunta"

In [ ]:
def answer_question(question):
    """
    Función para responder preguntas dadas una pregunta y un contexto predefinido.
    """
    # Obtener la respuesta del modelo
    result = nlp(question=question, context=contexto)

    if result["score"] > 0.79:
       # Verificar si se encontró una respuesta
       if result['answer']:  # Si la respuesta no está vacía
           # Retornar la respuesta encontrada
           return result['answer']
       else:
           # Devolver un mensaje indicando que la pregunta debe ser reformulada
           return "Lo siento, no pude encontrar una respuesta para tu pregunta. Por favor, reformula tu pregunta"

    else:
       # Devolver un mensaje indicando que la pregunta debe ser reformulada
       return "Lo siento, no score"

# Prueba de la función
print(answer_question("¿Qué puedo avisar al servicio de incidencias?"))

cortes de agua o fugas


In [ ]:
def answer_question(question):
    """
    Función para responder preguntas dadas una pregunta y un contexto predefinido.
    """
    # Obtener la respuesta del modelo
    result = nlp(question=question, context=contexto)

    if result["score"] > 0.79:
       # Verificar si se encontró una respuesta
       if result['answer']:  # Si la respuesta no está vacía
           # Retornar la respuesta encontrada
           return result['answer']
       else:
           # Devolver un mensaje indicando que la pregunta debe ser reformulada
           return "Lo siento, no pude encontrar una respuesta para tu pregunta. Por favor, reformula tu pregunta"

    else:
       # Devolver un mensaje indicando que la pregunta debe ser reformulada
       return "Lo siento, no score"

# Prueba de la función
print(answer_question("¿Qué cuatro conceptos hay en la factura?"))

Lo siento, no score


## Modelo preentrenado reducido (prueba) para responder preguntas dadas un contexto específico reducido (prueba), sobre los servicios de agua en la empresa Canal de Isabel II de suministro de agua

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

# Cargar el modelo y el tokenizador
model_name ='Antonio49/Personal'
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Configurar el pipeline para la pregunta-respuesta
nlp = pipeline('question-answering', model=model, tokenizer=tokenizer)

# Contexto integrado en el código
context = "En primer lugar, averigua si la falta de agua afecta a todos los puntos de agua de tu casa. Si no, pregunta a algún vecino si tiene agua. En caso de que sí tenga, comprueba que la llave de paso interior de tu vivienda esté completamente abierta y verifica que el contador o el equipo de medición tenga las dos llaves de paso abiertas (es decir, giradas completamente a la izquierda). En la batería de contadores hay un plano donde podrás encontrar la ubicación de tu contador. Si después de haber hecho estas comprobaciones sigues sin agua, ponte en contacto con nosotros. Comprueba que la llave de paso interior de tu casa esté completamente abierta. Si tienes instalado algún aparato reductor de presión, comprueba que funcione correctamente. A continuación, verifica que el contador o el equipo de medición tenga las dos llaves de paso abiertas (es decir, giradas completamente a la izquierda). Sí, hemos puesto en marcha un nuevo servicio para comunicarte de manera inmediata las incidencias que pudieran producirse en la red de distribución, bien por obras de mejora o por situaciones sobrevenidas, y la duración estimada de la suspensión temporal del suministro. Este servicio se ha desarrollado para que puedas adoptar las medidas necesarias para reducir el impacto que estas intervenciones pudieran ocasionar en tu actividad diaria. Ponte en contacto con nosotros a través del teléfono gratuito 900 365 365, o date de alta desde tu perfil (para ello deberás estar registrado). Durante la noche, cuando ya no se vaya a realizar consumo de agua, cierra todos los grifos y toma la lectura del contador. Al día siguiente, antes de abrir cualquier grifo, anota de nuevo la lectura. Si la lectura es diferente, es posible que exista una fuga en la instalación. Cierra todos los grifos: Comprueba que todos aquellos electrodomésticos que consumen agua no están en funcionamiento. Asegúrate también de que las llaves de paso anteriores y posteriores al contador están abiertas. Verifica que tu contador no registra consumo: Comprueba que las fracciones de metros cúbicos permanecen en la misma posición pasado un tiempo. Si observas un avance continuado de estos dígitos sin que se esté produciendo ningún consumo, es posible que exista una fuga. En tal caso, habrá que localizarla y repararla en el menor tiempo posible. En las zonas verdes, es recomendable conocer los caudales reales de riego de cada instalación. Para ello, puedes contabilizar el consumo registrado por el contador durante un tiempo determinado mientras tienes activado únicamente el riego. Este caudal te servirá de referencia para periódicamente verificar su valor. En caso de que, exclusivamente durante el riego, detectes un incremento injustificado de ese valor normal, será un indicador de la aparición de una fuga en la instalación de riego. En primer lugar, averigua si la falta de agua afecta a todos los puntos de agua de tu casa. Si no, pregunta a algún vecino si tiene agua. En caso de que sí tenga, comprueba que la llave de paso interior de tu vivienda esté completamente abierta y verifica que el contador o el equipo de medición tenga las dos llaves de paso abiertas (es decir, giradas completamente a la izquierda). En la batería de contadores hay un plano donde podrás encontrar la ubicación de tu contador. Si después de haber hecho estas comprobaciones sigues sin agua, ponte en contacto con nosotros. En numerosas ocasiones, las fugas interiores se deben a pequeños goteos en grifos, calderas o cisternas, por lo que es conveniente controlar periódicamente el consumo de estos. Para un mejor mantenimiento de tu instalación interior: 1.Observa que la cisterna del inodoro no tiene pérdidas: Hazlo una vez haya terminado el llenado de la cisterna. Además de la revisión visual y sonora, puedes añadir colorante alimentario en el tanque para que te resulte más fácil detectar una posible fuga. 2.Comprueba el estado de grifos y calderas: Vigila que no goteen. Verifica también que el circuito cerrado de la calefacción o la caldera funciona correctamente. A veces se producen fallos que hacen que el agua se pierda directamente por la red de saneamiento. Es recomendable que conozcas el trazado de las tuberías, así como la localización de las llaves de corte o seccionamiento. Accionando estas llaves se puede acotar el tramo de la instalación en el que se localiza la fuga para así facilitar las labores de reparación. Puedes elegir siete vías diferentes para contactar con nosotros: La manera más rápida es utilizar el chat de esta Oficina Virtual. Si prefieres hablar por teléfono, llama a nuestro teléfono gratuito 900 365 365 en horario de 08:00 a 20:00. Si necesitas comunicar una incidencia o avería, el servicio se encuentra operativo las 24 horas del día, los 365 días del año. Otra opción telefónica es que te llamemos nosotros. Solo tienes que indicar tu número de teléfono y el rango horario que prefieras. Puedes utilizar un formulario de contacto. Tienes la posibilidad de escribirnos un correo electrónico a la dirección: clientes@canaldeisabelsegunda.es. Puedes pedir cita para que te atendamos en nuestros centros de atención al cliente. Y si prefieres métodos tradicionales, puedes enviar una comunicación escrita por correo ordinario o fax: Att. Registro General. C/ Santa Engracia, 125 (28003 - Madrid). Horario: de lunes a viernes de 8:30 a 14:00 horas. Fax: 915 451 430. Independientemente del canal de comunicación que prefieras utilizar, no olvides indicarnos como referencia tu número de contrato o la dirección de la finca objeto de la solicitud, así como un teléfono de contacto. Si no eres usuario de la Oficina virtual puedes realizar las siguientes gestiones: 1.Relacionadas con contratación: Contratar un suministro ,darte de alta como cliente sensible y realizar gestiones asociadas con alcantarillado y saneamiento. 2.Relacionadas con tus facturas: Pagar tus facturas, visualizar facturas electrónicas, realizar simulaciones de facturas, solicitar bonificaciones y darte de alta en la factura electrónica. 3.Relativas a consumos y lecturas: Consultar la fecha de la próxima lectura y enviar la lectura de tu contador. 4.Otro tipo de gestiones: Cambiar la titularidad de un contrato, darte de alta en la Oficina virtual, pedir una cita para atención personalizada en una oficina, gestionar tus citas, comunicar incidencias, consultar solicitudes, acceder a la oficina de reclamaciones y notificar incidencias en el suministro. Si ya estás dado de alta en la Oficina Virtual, además de todas las gestiones recogidas en el apartado anterior, puedes acceder a las siguientes: 1.Trámites que afectan a tus contratos: Actualización de los datos de contacto y modificación de los datos de pago. 2.Asociados a tus facturas: Consultar facturas anteriores, reclamar alguna factura, enviarnos un justificante de pago, solicitar una devolución de saldo y descargar un listado de tus facturas. 3.Relacionadas con cancelaciones: Dar de baja un suministro y darte de baja como usuario de la Oficina virtual. 4.Otras gestiones: Consultar el estado de tus quejas y acceder al arbitraje de consumo. En tu factura podrás distinguir cuatro conceptos facturados: aducción, distribución, depuración y alcantarillado. Cada uno de ellos se ve afectado por unas ratios diferentes y consta de dos partes. Por un lado, una cuota de servicio que es fija y se factura independientemente de que exista o no consumo. Por otro, una parte variable que dependerá del consumo de agua realizado en el periodo que se factura. Bimestralmente. Es decir, cada dos meses. Una vez emitida la factura, tienes un periodo voluntario de pago de 30 días naturales. El Reglamento para el Servicio y Distribución de las Aguas de Canal de Isabel II (Decreto 2922/1975, de 31 de octubre) establece varios métodos para facturar el consumo de agua: 1.Estimación de consumos (EST): este método se utiliza cuando no es posible acceder al contador en la fecha en que debe realizarse la lectura. Su cálculo se efectúa de acuerdo con el consumo medio diario de los dos periodos análogos precedentes y, en caso de disponerse solo del histórico de consumos de un año, la estimación se realizará de acuerdo con el consumo medio diario del periodo análogo precedente. El consumo facturado de esta forma se considera a cuenta. Por tanto, una vez que se tome la lectura real del contador, los metros cúbicos facturados de esta forma se descontarán del consumo realizado. 2.Evaluación de consumos (EV): para facturar los metros cúbicos mediante este método, se debe dar la circunstancia de que, aun teniendo acceso al contador, no se pueda recoger la lectura porque exista una anomalía que requiera su sustitución. El cálculo del consumo se realiza de forma análoga a la estimación. 3.Diferencia de lecturas del aparato de medida (DFI): diferencia de la lectura tomada en el periodo anterior y la actual. Puedes consultar las tarifas aplicables para la facturación del servicio de suministro de agua en la sección de tarifas de esta web.  Estudiamos individualmente las solicitudes, y tras su análisis, concedemos, de acuerdo con una serie de criterios, el número de plazos e interés correspondiente según el importe y las características del cliente. Según establece el Reglamento para el Servicio y Distribución de las Aguas de Canal de Isabel II, (Decreto 2922/1975, de 31 de octubre) podríamos suspender el suministro de agua si transcurridos 30 días naturales desde la emisión de la factura, esta no ha sido abonada. El corte del suministro implicaría continuar facturando las cuotas de servicio. El restablecimiento del servicio se realizará una vez liquidada la deuda, así como el importe del restablecimiento. Transcurridos tres meses sin que se haya producido el pago, Canal puede resolver el contrato y proceder a la condena de la acometida (retirada de la instalación). Para volver a tener agua habrá que realizar una nueva contratación."

In [ ]:
def answer_question(question):
    """
    Función para responder preguntas dadas una pregunta y un contexto predefinido.
    """
    # Obtener la respuesta del modelo
    result = nlp(question=question, context=context)

    # Verificar si se encontró una respuesta
    if result['answer']:  # Si la respuesta no está vacía
        # Retornar la respuesta encontrada
        return result['answer']
    else:
        # Devolver un mensaje indicando que la pregunta debe ser reformulada
        return "Lo siento, no pude encontrar una respuesta para tu pregunta. Por favor, reformula tu pregunta"

En este código, la función verificará si la respuesta devuelta por el modelo no está vacía. Si la respuesta está vacía, la función devolverá un mensaje indicando que no pudo encontrar una respuesta para la pregunta.






In [ ]:
question = "¿Cada cuánto tiempo recibo mi factura?"
answer_question(question)

'cada dos meses'

In [ ]:
question = "¿Cuanto es el tiempo voluntario de pago?"
answer_question(question)

'30 días naturales'

In [ ]:
question = "¿Cuál es la dirección de correo electrónico de Canal de Isabel II?"
answer_question(question)

'900 365 365'

**Aqui probamos si supera un nivel de score**

In [ ]:
def answer_question(question):
    """
    Función para responder preguntas dadas una pregunta y un contexto predefinido.
    """
    # Obtener la respuesta del modelo
    result = nlp(question=question, context=context)

    if result["score"] > 0.2:
       # Verificar si se encontró una respuesta
       if result['answer']:  # Si la respuesta no está vacía
           # Retornar la respuesta encontrada
           return result['answer']
       else:
           # Devolver un mensaje indicando que la pregunta debe ser reformulada
           return "Lo siento, no pude encontrar una respuesta para tu pregunta. Por favor, reformula tu pregunta"

    else:
       # Devolver un mensaje indicando que la pregunta debe ser reformulada
       return "Lo siento, no score"


In [ ]:
question = "¿Cuanto es el tiempo voluntario de pago?"
answer_question(question)

'30 días naturales'

In [ ]:
resp = nlp(context=context, question=question)
resp

{'score': 0.20767243206501007,
 'start': 7623,
 'end': 7640,
 'answer': '30 días naturales'}

In [ ]:
question = "¿Que puedes responderme?"
answer_question(question)

'Lo siento, no score'

In [ ]:
resp = nlp(context=context, question=question)
resp

{'score': 0.05990104749798775,
 'start': 8998,
 'end': 9040,
 'answer': 'Estudiamos individualmente las solicitudes'}

In [ ]:
question = " "
answer_question(question)

'Lo siento, no score'

In [ ]:
resp = nlp(context=context, question=question)
resp

{'score': 0.0022760480642318726,
 'start': 1485,
 'end': 1492,
 'answer': 'Durante'}

In [ ]:
question = "¿Cada cuánto tiempo recibo mi factura?"
answer_question(question)

'Lo siento, no score'

In [ ]:
resp = nlp(context=context, question=question)
resp

{'score': 0.15548183023929596,
 'start': 7539,
 'end': 7553,
 'answer': 'cada dos meses'}